In [ ]:
%reset -f

In [ ]:
# load dependencies
###################
import pyvista as pv

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

import numpy as np
np.bool = np.bool_

import cmocean

import glob

import os

import itertools

import trame

import ipywidgets

from sklearn import preprocessing

from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

from numpy import loadtxt

pv.set_jupyter_backend('ipyvtklink')

In [ ]:
### load participant IDs ###
lines = loadtxt("dummy_data/one_hander/all_IDs.txt", dtype=str, comments="#", delimiter=" ", unpack=False)
print(lines)
subjects = lines

### specifiy folder structure of MIPAV results (need to match order of participant IDs) ###
subject_folder = ['00','01']
#print(subject_folder)

#### define the colors we want to use to visualise the different fingers ###
red = np.array([1, 0, 0, 1])
magenta = np.array([1, 0, 1, 1])
blue = np.array([0, 0, 1, 1])
green = np.array([0, 1, 0, 1])
yellow = np.array([255/256, 247/256, 0/256, 1])
grey = np.array([0.75, 0.75, 0.75, 1])
white = np.array([1, 1, 1, 1])
black = np.array([0, 0, 0, 1])
olive = np.array([232/256, 241/256, 177/256, 1])

folder = 0

In [ ]:
# change ID and folder for subject to analyze here: #
subj = 0

ind = subjects[subj]
print(ind)
subject_folder = [subject_folder[subj]]
print(subject_folder)
ind_i = subj

error_msg = "All paths successfully calculated."

In [ ]:
lightpink = np.array([256 / 256, 128 / 256, 178 / 256, 1.0])
cyan = np.array([1 / 256, 256 / 256, 256 / 256, 1.0])

In [ ]:
#### load face localizers here ###
##################################
try:
    if subj == 0:
        folder_spmT_f = 'dummy_data/one_hander/'+ind+'_face_win_thrs_left_orig_70.vtk'
        print(folder_spmT_f)
    else:
        folder_spmT_f = 'dummy_data/one_hander/'+ind+'_face_win_thrs_right_orig_70.vtk'

    # initialize mesh #
    mesh_f = pv.read(folder_spmT_f)

    # initilaize color mapping #
    mapping_f = np.linspace(mesh_f['EmbedVertex'].min(), mesh_f['EmbedVertex'].max(), 256)
    newcolors_f = np.empty((256, 4))
    newcolors_f[mapping_f > 0.01] = cyan
    newcolors_f[mapping_f < 0.01] = white

    my_colormap_f = ListedColormap(newcolors_f)
except:
    print("No face localizer.")
    
### load hand localizers here ###
#################################
try:
    if subj == 0:
        folder_spmT_h = 'dummy_data/one_hander/'+ind+'_hand_win_thrs_left_orig_70.vtk'
    else:
        folder_spmT_h = 'dummy_data/one_hander/'+ind+'_hand_win_thrs_right_orig_70.vtk'
    #os.chdir(folder_spmT_h)
    #filename_spmT_h = glob.glob('*_hand_S1_smoothdata.vtk')

    # initilaize mesh #
    mesh_h = pv.read(folder_spmT_h)

    # intialize color mapping #
    mapping_h = np.linspace(mesh_h['EmbedVertex'].min(), mesh_h['EmbedVertex'].max(), 256)
    newcolors_h = np.empty((256, 4))
    newcolors_h[mapping_h > 0] = lightpink
    newcolors_h[mapping_h < 0] = white

    my_colormap_h = ListedColormap(newcolors_h)
except:
    print("No hand localizer.")

### load qT1 Maps ###
#####################
folder_qT1 = 'dummy_data/one_hander/'
smooth_folder_qT1 = 'dummy_data/one_hander/'

os.chdir(folder_qT1)
if subj == 0:
    filename_qT1_dp = subjects[ind_i]+'_qT1_S1_inner_DDlayers_left_orig.vtk'
    filename_qT1_im = subjects[ind_i]+'_qT1_S1_middle_DDlayers_left_orig.vtk'
    filename_qT1_om = subjects[ind_i]+'_qT1_S1_outer_DDlayers_left_orig.vtk'
else:
    filename_qT1_dp = subjects[ind_i]+'_qT1_S1_inner_DDlayers_right_orig.vtk'
    filename_qT1_im = subjects[ind_i]+'_qT1_S1_middle_DDlayers_right_orig.vtk'
    filename_qT1_om = subjects[ind_i]+'_qT1_S1_outer_DDlayers_right_orig.vtk'
    
os.chdir(smooth_folder_qT1)
smooth_filename_qT1_im = glob.glob('*_smoothdata.vtk')

# read in poly data #
mesh_qT1_dp = pv.read(folder_qT1+filename_qT1_dp)
mesh_qT1_im = pv.read(folder_qT1+filename_qT1_im)
smooth_mesh_qT1_im = pv.read(smooth_folder_qT1+smooth_filename_qT1_im[0])
mesh_qT1_om = pv.read(folder_qT1+filename_qT1_om)

### load full data array with spmT finger peak values ###
#########################################################

import scipy.io

if subj==0:
    filename_qT1_S1_full_dict = scipy.io.loadmat('dummy_data/one_hander/'+ind+'_body_part_win_S1_thrs_left_orig_70.mat')
    tmp_hand = scipy.io.loadmat('dummy_data/one_hander/'+ind+'_T1_mhand_DDlayer_thrs_left_orig_70.mat')
    tmp_face = scipy.io.loadmat('dummy_data/one_hander/'+ind+'_T1_mface_DDlayer_thrs_left_orig_70.mat')
    
    filename_qT1_S1_full = filename_qT1_S1_full_dict['finger_winner_S1']
    print(filename_qT1_S1_full)
    
    print(tmp_hand)
    tmp_handvals = tmp_hand['T1_D1']
    print(tmp_handvals)
    
    print(tmp_face)
    tmp_facevals = tmp_face['T1_D2']
    
    filename_qT1_S1_full = np.concatenate((filename_qT1_S1_full,tmp_handvals,tmp_facevals))
    print(filename_qT1_S1_full)
    
    filename_d1 = 'dummy_data/one_hander/'+ind+'_hand_win_thrs_left_orig_70.vtk'
    filename_d2 = 'dummy_data/one_hander/'+ind+'_face_win_thrs_left_orig_70.vtk'
else:
    filename_qT1_S1_full_dict = scipy.io.loadmat('dummy_data/one_hander/'+ind+'_body_part_win_S1_thrs_right_orig_70.mat')
    tmp_hand = scipy.io.loadmat('dummy_data/one_hander/'+ind+'_T1_mhand_DDlayer_thrs_right_orig_70.mat')
    tmp_face = scipy.io.loadmat('dummy_data/one_hander/'+ind+'_T1_mface_DDlayer_thrs_right_orig_70.mat')
    
    filename_qT1_S1_full = filename_qT1_S1_full_dict['finger_winner_S1']
    print(filename_qT1_S1_full)
    
    print(tmp_hand)
    tmp_handvals = tmp_hand['T1_D1']
    print(tmp_handvals)
    
    print(tmp_face)
    tmp_facevals = tmp_face['T1_D2']
    
    filename_qT1_S1_full = np.concatenate((filename_qT1_S1_full,tmp_handvals,tmp_facevals))
    print(filename_qT1_S1_full)
    
    filename_d1 = 'dummy_data/one_hander/'+ind+'_hand_win_thrs_right_orig_70.vtk'
    filename_d2 = 'dummy_data/one_hander/'+ind+'_face_win_thrs_right_orig_70.vtk'

# read in file that contains extracted values after applying winner-takes-all approach
qT1_S1_full = filename_qT1_S1_full

#read in pRF surfaces for plotting
##################################

mesh_d1 = pv.read(filename_d1)
mesh_d2 = pv.read(filename_d2)

### specify results folder ###
##############################
if subj==0:
    outdir='dummy_data/one_hander/results/septa/v8/'
    outdir_all='dummy_data/one_hander/results/septa/v8/'
else:
    outdir='dummy_data/one_hander/results/septa/v8/right'
    outdir_all='dummy_data/one_hander/results/septa/v8/'

In [ ]:
# generate binary BA3b mask #
tmp = mesh_qT1_im
tmp0 = tmp['EmbedVertex']
tmp0[tmp0>0] = 1
tmp['EmbedVertex'] = tmp0

# apply binary BA3b mask to smoothed maps #
tmp1=smooth_mesh_qT1_im
tmp2=tmp1['EmbedVertex']*tmp['EmbedVertex']
#tmp1['EmbedVertex']=tmp2
smooth_mesh_qT1_im['EmbedVertex']=tmp2

In [ ]:
# extract mean and sd from qT1 surface
######################################
vtx_mesh_qT1_im = smooth_mesh_qT1_im['EmbedVertex']

print(vtx_mesh_qT1_im.max())

print(vtx_mesh_qT1_im[np.nonzero(vtx_mesh_qT1_im)])

vtx_mesh_qT1_im_omitzero = vtx_mesh_qT1_im[np.nonzero(vtx_mesh_qT1_im)]

mean_im = vtx_mesh_qT1_im_omitzero.mean()
sd_im = vtx_mesh_qT1_im_omitzero.std()

print(mean_im)
print(sd_im)

In [ ]:
# plot smoothed qT1 values restricted to BA3b
#############################################
qT1_cmap = cmocean.cm.turbid_r
if subj==0:
    p4_camera_position = [(235.22052780809193, 39.8387107829154, -139.23390663890612),(23.55722029834093, 79.19966881614376, 65.00208781014945),(-0.6493875573065231, -0.494533993462862, -0.57769536065745)]
else:
    p4_camera_position = [(-241.809, 8.5008, -79.353),(29.1759, 82.2512, 61.8365),(0.86209, -0.234624, -0.449169)]
p = pv.Plotter()
#p.add_mesh(tmp1, cmap=qT1_cmap, clim=[mean_dp-(4*sd_dp), mean_dp+(4*sd_dp)],below_color="#ffffff")
p.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p.camera_position = p4_camera_position
p.show()

## should you experience 'vtkxopenglrenderwindow could not find a decent config' error at this stage, shut down jupyter notebooks and type the following command in your terminal:
## export LD_PRELOAD='/usr/lib/x86_64-linux-gnu/libstdc++.so.6'
## restart jupyter notebook and run again from the beginning

In [ ]:
print(p.camera)

In [ ]:
# extract mean and sd from qT1 surface
######################################
mesh_qT1_dp = pv.read(folder_qT1+filename_qT1_dp)
vtx_mesh_qT1_dp = mesh_qT1_dp['EmbedVertex']

print(vtx_mesh_qT1_dp.max())

print(vtx_mesh_qT1_dp[np.nonzero(vtx_mesh_qT1_dp)])

vtx_mesh_qT1_dp_omitzero = vtx_mesh_qT1_dp[np.nonzero(vtx_mesh_qT1_dp)]

mean_dp = vtx_mesh_qT1_dp_omitzero.mean()
sd_dp = vtx_mesh_qT1_dp_omitzero.std()

print(mean_dp)
print(sd_dp)

In [ ]:
mesh_qT1_im = pv.read(folder_qT1+filename_qT1_im)

In [ ]:
# initialize color mapping
##########################
mapping_bg = np.linspace(mesh_qT1_dp['EmbedVertex'].min(), mesh_qT1_dp['EmbedVertex'].max(), 256)
newcolors_bg = np.empty((256, 4))
newcolors_bg[mapping_bg >= 0] = white

# pRF maps #
mapping_d1 = np.linspace(mesh_d1['EmbedVertex'].min(), mesh_d1['EmbedVertex'].max(), 256)
newcolors = np.empty((256, 4))
newcolors[mapping_d1 > 0] = red
newcolors[mapping_d1 <= 0] = white

mapping_d2 = np.linspace(mesh_d2['EmbedVertex'].min(), mesh_d2['EmbedVertex'].max(), 256)
newcolors_d2 = np.empty((256, 4))
newcolors_d2[mapping_d2 > 0] = magenta
newcolors_d2[mapping_d2 <= 0] = white

#mapping_d3 = np.linspace(mesh_d3['EmbedVertex'].min(), mesh_d3['EmbedVertex'].max(), 256)
#newcolors_d3 = np.empty((256, 4))
#newcolors_d3[mapping_d3 > 0] = blue
#newcolors_d3[mapping_d3 <= 0] = white

#mapping_d4 = np.linspace(mesh_d4['EmbedVertex'].min(), mesh_d4['EmbedVertex'].max(), 256)
#newcolors_d4 = np.empty((256, 4))
#newcolors_d4[mapping_d4 > 0] = green
#newcolors_d4[mapping_d4 <= 0] = white

#mapping_d5 = np.linspace(mesh_d5['EmbedVertex'].min(), mesh_d5['EmbedVertex'].max(), 256)
#newcolors_d5 = np.empty((256, 4))
#newcolors_d5[mapping_d5 > 0] = yellow
#newcolors_d5[mapping_d5 <= 0] = white

# spmT maps #
#mapping_spmT_d1 = np.linspace(0, mesh_spmT_d1['EmbedVertex'].max(), 256)
#newcolors_spmT = np.empty((256, 4))
#newcolors_spmT[mapping_spmT_d1 > 0] = red
#newcolors_spmT[mapping_spmT_d1 <= 0] = white

#mapping_spmT_d2 = np.linspace(0, mesh_spmT_d2['EmbedVertex'].max(), 256)
#newcolors_spmT_d2 = np.empty((256, 4))
#newcolors_spmT_d2[mapping_spmT_d2 > 0] = magenta
#newcolors_spmT_d2[mapping_spmT_d2 <= 0] = white

#mapping_spmT_d3 = np.linspace(0, mesh_spmT_d3['EmbedVertex'].max(), 256)
#newcolors_spmT_d3 = np.empty((256, 4))
#newcolors_spmT_d3[mapping_spmT_d3 > 0] = blue
#newcolors_spmT_d3[mapping_spmT_d3 <= 0] = white

#mapping_spmT_d4 = np.linspace(0, mesh_spmT_d4['EmbedVertex'].max(), 256)
#newcolors_spmT_d4 = np.empty((256, 4))
#newcolors_spmT_d4[mapping_spmT_d4 > 0] = green
#newcolors_spmT_d4[mapping_spmT_d4 <= 0] = white

#mapping_spmT_d5 = np.linspace(0, mesh_spmT_d5['EmbedVertex'].max(), 256)
#newcolors_spmT_d5 = np.empty((256, 4))
#newcolors_spmT_d5[mapping_spmT_d5 > 0] = yellow
#newcolors_spmT_d5[mapping_spmT_d5 <= 0] = white

# make the colormap from the listed colors #
my_colormap_bg = ListedColormap(newcolors_bg)

#my_colormap_spmT_d1 = ListedColormap(newcolors_spmT)
#my_colormap_spmT_d2 = ListedColormap(newcolors_spmT_d2)
#my_colormap_spmT_d3 = ListedColormap(newcolors_spmT_d3)
#my_colormap_spmT_d4 = ListedColormap(newcolors_spmT_d4)
#my_colormap_spmT_d5 = ListedColormap(newcolors_spmT_d5)

my_colormap_d1 = ListedColormap(newcolors)
my_colormap_d2 = ListedColormap(newcolors_d2)
#my_colormap_d3 = ListedColormap(newcolors_d3)
#my_colormap_d4 = ListedColormap(newcolors_d4)
#my_colormap_d5 = ListedColormap(newcolors_d5)

# load colormap for qT1 Maps #
qT1_cmap = cmocean.cm.turbid_r

# load color map for hand / face localizers # 
body_part = plt.get_cmap('PuOr')

In [ ]:
# plot hand localizer
#####################
#if ind_i == 39:
#    p4_camera_position = [(235.22052780809193, 39.8387107829154, -139.23390663890612),(33.55722029834093, 69.19966881614376, 75.00208781014945),(-0.8493875573065231, -0.694533993462862, -0.57769536065745)]
#else:
if subj==0:
    p4_camera_position = [(235.22052780809193, 39.8387107829154, -139.23390663890612),(23.55722029834093, 79.19966881614376, 65.00208781014945),(-0.6493875573065231, -0.494533993462862, -0.57769536065745)]
else:
    p4_camera_position = [(-241.809, 8.5008, -79.353),(29.1759, 75.2512, 52.8365),(0.86209, -0.234624, -0.449169)]

p = pv.Plotter()
p.set_background('white')
try:
    p.add_mesh(mesh_h, cmap="RdPu")
except:
    print("No hand localizer.")
p.camera_position = p4_camera_position
p.show()

In [ ]:
# plot face localizer
#####################
#p4_camera_position = [(235.22052780809193, 39.8387107829154, -139.23390663890612),(23.55722029834093, 79.19966881614376, 65.00208781014945),(-0.6493875573065231, -0.494533993462862, -0.57769536065745)]
p = pv.Plotter()
try:
    p.add_mesh(mesh_f, cmap='RdPu')
except:
    print("No face localizer.")
p.camera_position = p4_camera_position
p.show()

In [ ]:
# extract different contrast values and store them into one common mesh
#######################################################################
mesh_qT1_dp_orig = pv.read(folder_qT1+filename_qT1_dp)

try:
    mesh_qT1_dp['Hand'] = mesh_h['EmbedVertex']
    mesh_qT1_dp['Face'] = mesh_f['EmbedVertex']
except:
    print("No hand/face localizer.")
    
#mesh_qT1_dp['D1_spmT'] = mesh_spmT_d1['EmbedVertex']
#mesh_qT1_dp['D2_spmT'] = mesh_spmT_d2['EmbedVertex']
#mesh_qT1_dp['D5_spmT'] = mesh_spmT_d5['EmbedVertex']
#mesh_qT1_dp['D1b'] = qT1_S1_full[22]
#mesh_qT1_dp['D2b'] = qT1_S1_full[23]
#mesh_qT1_dp['D5b'] = qT1_S1_full[26]

mesh_qT1_dp['HspmT'] = qT1_S1_full[1]
mesh_qT1_dp['FspmT'] = qT1_S1_full[2]
mesh_qT1_dp['HmqT1'] = qT1_S1_full[5]
mesh_qT1_dp['FmqT1'] = qT1_S1_full[8]
mesh_qT1_dp['middleqT1'] = mesh_qT1_im['EmbedVertex']
mesh_qT1_dp['smoothmiddleqT1'] = smooth_mesh_qT1_im['EmbedVertex']
mesh_qT1_dp['qT1'] = mesh_qT1_dp_orig['EmbedVertex']

# reduce mesh to valid vertices (i.e. with vertex value > 0)
############################################################
vertex_values = mesh_qT1_dp['EmbedVertex']
vertex_values[vertex_values<1]=0
invalid_vertices = np.where(vertex_values == 0)[0]

threshed = mesh_qT1_dp.remove_points(invalid_vertices)

print(threshed)
print(threshed[0]['qT1'])
print(len(threshed[0]['EmbedVertex']))

# find location of activation peaks
###################################
spmT_h_full = threshed[0]['Hand'].argmax()
print(spmT_h_full)

spmT_f_full = threshed[0]['Face'].argmax()

avg_v2v_dist = 0.28

distance_f_h = 0

distance_d2_d5 = 0

# define function that finds the face seed 
##########################################

def get_face_seed(threshed, spmT_h_full, multiplicator,dist):
    zf = threshed[0].points[spmT_h_full,2]
    yf = threshed[0].points[spmT_h_full,1]
    xf = threshed[0].points[spmT_h_full,0]
    #zf_min = zf-(avg_v2v_dist*multiplicator)
    #zf_max = zf+(avg_v2v_dist*multiplicator)
    yf_min = yf-(avg_v2v_dist*multiplicator)
    yf_max = yf+(avg_v2v_dist*multiplicator)
    xf_min = xf-(avg_v2v_dist*multiplicator)
    xf_max = xf+(avg_v2v_dist*multiplicator)
    #print(zf_max)
    #print(zf_min)

    zf_multi = np.where((threshed[0].points[:,0]<xf_max) & (threshed[0].points[:,0]>xf_min) & (threshed[0].points[:,1]<yf_max) & (threshed[0].points[:,1]>yf_min) & (threshed[0].points[:,2]>zf) & (threshed[0]['qT1']>1000))
    print(zf_multi[0])

    distance = []

    for zfm in zf_multi[0]:
        #print(zfm)
        #print(spmT_h_full)
        #print('test')
        distance.append(threshed[0].geodesic_distance(zfm, spmT_h_full))

    distance_diff = abs(np.array(distance) - dist)
    distance_idx = distance_diff.argmin()

    spmT_f_full = zf_multi[0][distance_idx]

    #print(spmT_f_full)
    
    distance_f_h = threshed[0].geodesic_distance(spmT_f_full, spmT_h_full)
    
    print(distance_f_h)     
    
    return zf_multi, spmT_f_full, distance_f_h

# define function that finds a seed near the superior border of the hand representation (where the little finger would be located)
#################################################################################################################################

def get_d5_seed(threshed, spmT_h_full, multiplicator, dist):
    zf = threshed[0].points[spmT_h_full,2]
    yf = threshed[0].points[spmT_h_full,1]
    xf = threshed[0].points[spmT_h_full,0]
    #zf_min = zf-(avg_v2v_dist*multiplicator)
    #zf_max = zf+(avg_v2v_dist*multiplicator)
    yf_min = yf-(avg_v2v_dist*multiplicator)
    yf_max = yf+(avg_v2v_dist*multiplicator)
    xf_min = xf-(avg_v2v_dist*multiplicator)
    xf_max = xf+(avg_v2v_dist*multiplicator)
    #print(zf_max)
    #print(zf_min)

    zf_multi = np.where((threshed[0].points[:,0]<xf_max) & (threshed[0].points[:,0]>xf_min) & (threshed[0].points[:,1]<yf_max) & (threshed[0].points[:,1]>yf_min) & (threshed[0].points[:,2]<zf) & (threshed[0]['qT1']>1000))
    print(zf_multi[0])

    distance = []

    for zfm in zf_multi[0]:
        #print(zfm)
        #print(spmT_h_full)
        #print('test')
        distance.append(threshed[0].geodesic_distance(spmT_h_full, zfm))

    distance_diff = abs(np.array(distance) - dist)
    distance_idx = distance_diff.argmin()

    #spmT_d5_full = zf_multi[0][distance_idx]

    #print(spmT_f_full)
    
    distance_d5_d5new = threshed[0].geodesic_distance(spmT_h_full, zf_multi[0][distance_idx])
    
    print(distance_d5_d5new)
    
    spmT_d5_full_old = spmT_h_full
    
    spmT_d5_full = zf_multi[0][distance_idx]
    
    return zf_multi, spmT_d5_full, distance_d5_d5new
    

In [ ]:
# call function to define face seed and set distance for geodesic path
######################################################################

# the max distance was set to 15 mm to ensure that the path (starting at the activation peak of the hand representation) reaches the superior face representation (here the face representation was localized by tongue movements)
# otherwise the hand-face border may not have been covered (forehead lies appr. 10 mm inferior to the thumb representation)

# needs to be optimized in future versions to target the forehead area more precisely (e.g. extracting the inferior border of the hand representation and sampling precisely 10 mm in inferior direction, or using a forehead localizer)

dist = 15
    
multiplicator = dist

while distance_f_h < dist-0.1:
    print('testtest')
    zf_multi, spmT_f_full, distance_f_h = get_face_seed(threshed, spmT_h_full, multiplicator, dist)
    multiplicator = multiplicator + dist
    if 0 in zf_multi[0]:
        dist = distance_f_h

if subj==1:
    dist = 17.5
    
    multiplicator = dist
    
    spmT_h_full = spmT_f_full
    print(spmT_h_full)
    
    spmT_d5_full = threshed[0]['Hand'].argmax()
    print(spmT_d5_full)
    
    distance_f_h = 0
    
    while distance_f_h < dist-0.1:
        print('testtest')
        zf_multi, spmT_f_full, distance_f_h = get_face_seed(threshed, spmT_h_full, multiplicator, dist)
        multiplicator = multiplicator + dist
        if 0 in zf_multi[0]:
            dist = distance_f_h


In [ ]:
# call function to define the seed at the superior hand map border and set distance for geodesic path
#####################################################################################################

# the max distance was set to 7 mm to ensure that the path (starting at the activation peak of the hand representation) reaches the superior part of the hand representation (approximating the superior border)
# otherwise the hand representation may not have been fully covered

# needs to be optimized (e.g. using xyz coordinates in combination with statistical values to find the superior border of the hand representation)

dist = 7
    
multiplicator = dist

if subj==0:
    while distance_d2_d5 < dist-0.1:
        print('testtest')
        zf_d5_multi, spmT_d5_full, distance_d2_d5 = get_d5_seed(threshed, spmT_h_full, multiplicator, dist)
        multiplicator = multiplicator + dist
        if 0 in zf_d5_multi[0]:
            dist = distance_d2_d5

In [ ]:
# extract geodesic paths between activation peaks
#################################################
path_f_h_full = threshed[0].geodesic(spmT_f_full, spmT_h_full)
distance_f_h = threshed[0].geodesic_distance(spmT_f_full, spmT_h_full)
print(distance_f_h)

path_d2_d5_full = threshed[0].geodesic(spmT_h_full, spmT_d5_full)
distance_d2_d5 = threshed[0].geodesic_distance(spmT_h_full, spmT_d5_full)
print(distance_d2_d5)

#path_d2_d5_full_old = threshed[0].geodesic(spmT_d2_full, spmT_d5_full_old)
#distance_d2_d5_old = threshed[0].geodesic_distance(spmT_d2_full, spmT_d5_full_old)
#print(distance_d2_d5_old)

# extract deep qT1 values along geodesic paths
##############################################
qT1_dp_path_f_h_full = threshed[0]['qT1'][path_f_h_full['vtkOriginalPointIds']]
qT1_dp_path_d2_d5_full = threshed[0]['qT1'][path_d2_d5_full['vtkOriginalPointIds']]


In [ ]:
# extract start and end points of geodesic paths (activation peaks of functional localizers)
############################################################################################
extracted_f = path_f_h_full.extract_points(0, adjacent_cells=False, include_cells=False)
extracted_h = path_d2_d5_full.extract_points(0, adjacent_cells=False, include_cells=False)
extracted_d5 = path_d2_d5_full.extract_points(len(path_d2_d5_full['vtkOriginalPointIds'])-1, adjacent_cells=False, include_cells=False)
#extracted_d5_old = path_d2_d5_full_old.extract_points(len(path_d2_d5_full_old['vtkOriginalPointIds'])-1, adjacent_cells=False, include_cells=False)

print(extracted_f)
print(extracted_h)
print(extracted_d5)

merged_path = path_f_h_full.merge(path_d2_d5_full)
print(merged_path)

In [ ]:
# extract geodesic distances from face to D5
################################################################################
distance_f_h_all = [0]

distance_d2_d5_all = [0]

i = 0

for i in range(len(path_f_h_full['vtkOriginalPointIds'])-1):
    cur_dist = threshed[0].geodesic_distance(path_f_h_full['vtkOriginalPointIds'][0], path_f_h_full['vtkOriginalPointIds'][i+1])
    distance_f_h_all.append(cur_dist)

print(len(distance_f_h_all))

for i in range(len(path_d2_d5_full['vtkOriginalPointIds'])-1):
    cur_dist = threshed[0].geodesic_distance(path_d2_d5_full['vtkOriginalPointIds'][0], path_d2_d5_full['vtkOriginalPointIds'][i+1])
    distance_d2_d5_all.append(cur_dist)

print(len(distance_d2_d5_all))

distance_d2_d5_all_fin = distance_d2_d5_all 

print(len(distance_d2_d5_all_fin))
print(distance_d2_d5_all_fin)

# calculate average vertex-to-vertex distance of individual participant
distance_f_h_all_diff = np.sum(np.diff(distance_f_h_all))/len(np.diff(distance_f_h_all))
print(distance_f_h_all_diff)

In [ ]:
# extract vertex from face localizer that is located 10 mm away from D1 peak along the shortest path between D1 peak and Face peak
####################################################################################################################
# this is important to approxiamte the location of the forehead within the face map which neighbors the thumb representation 
distance_f_h_all_rev = np.array(distance_f_h_all) - np.max(distance_f_h_all)

extracted_fd10 = path_f_h_full.extract_points(0, adjacent_cells=False, include_cells=False)
print(extracted_fd10)
#print(path_f_h_full['vtkOriginalPointIds'][dist_10])

extracted_fd10_idx = list(range(0,len(distance_f_h_all)+1))
print(extracted_fd10_idx)
print(path_f_h_full)

path_d1_dist10 = path_f_h_full.extract_points(extracted_fd10_idx,adjacent_cells=False,include_cells=False)
path_d1_dist10_origIDs = path_f_h_full['vtkOriginalPointIds'][0:len(distance_f_h_all)+1]
#print(path_d1_dist10)
print(path_f_h_full['vtkOriginalPointIds'])
print(path_d1_dist10_origIDs)

distance_f_h_all_rev_d10 = distance_f_h_all_rev[0:len(distance_f_h_all)+1]
print(distance_f_h_all_rev_d10)

spmT_f = path_f_h_full['vtkOriginalPointIds'][0]
print(spmT_f)
#print(spmT_f_full)

In [ ]:
# extract multiple start and end points
#######################################
# thumb
yh = threshed[0].points[spmT_h_full,1]
xh = threshed[0].points[spmT_h_full,0]
print(yh)
yh_min = yh-(avg_v2v_dist/2)
yh_max = yh+(avg_v2v_dist/2)
xh_min = xh-(avg_v2v_dist/2)
xh_max = xh+(avg_v2v_dist/2)
print(yh_max)
print(xh_max)
yh_multi = np.where((threshed[0].points[:,1]>yh_min) & (threshed[0].points[:,1]<yh_max) & (threshed[0]['qT1']>1000))
xh_multi = np.where((threshed[0].points[:,0]>xh_min) & (threshed[0].points[:,0]<xh_max) & (threshed[0]['qT1']>1000))
print(yh_multi)
print(xh_multi)
print(len(yh_multi[0]))
print(len(xh_multi[0]))

yh_multi_yval = np.argsort(threshed[0].points[yh_multi[0],0])
yh_multi_yidx = np.take_along_axis(yh_multi[0],yh_multi_yval,axis=0)
print(yh_multi_yval)
print(yh_multi_yidx)

xh_multi_yval = np.argsort(threshed[0].points[xh_multi[0],1])
xh_multi_yidx = np.take_along_axis(xh_multi[0],xh_multi_yval,axis=0)
print(xh_multi_yval)
print(xh_multi_yidx)

test_yh = threshed[0].extract_points(yh_multi,adjacent_cells=False, include_cells=False)
test_xh = threshed[0].extract_points(xh_multi,adjacent_cells=False, include_cells=False)

# face
yf = threshed[0].points[spmT_f,1]
xf = threshed[0].points[spmT_f,0]
print(yf)
yf_min = yf-(avg_v2v_dist/2)
yf_max = yf+(avg_v2v_dist/2)
xf_min = xf-(avg_v2v_dist/2)
xf_max = xf+(avg_v2v_dist/2)
print(yf_max)
print(xf_max)
yf_multi = np.where((threshed[0].points[:,1]>yf_min) & (threshed[0].points[:,1]<yf_max) & (threshed[0]['qT1']>1000))
xf_multi = np.where((threshed[0].points[:,0]>xf_min) & (threshed[0].points[:,0]<xf_max) & (threshed[0]['qT1']>1000))
print(yf_multi[0])
print(xf_multi[0])

yf_multi_yval = np.argsort(threshed[0].points[yf_multi[0],0])
yf_multi_yidx = np.take_along_axis(yf_multi[0],yf_multi_yval,axis=0)
print(yf_multi_yval)
print(yf_multi_yidx)

xf_multi_yval = np.argsort(threshed[0].points[xf_multi[0],1])
xf_multi_yidx = np.take_along_axis(xf_multi[0],xf_multi_yval,axis=0)
print(xf_multi_yval)
print(xf_multi_yidx)

test_yf = threshed[0].extract_points(yf_multi,adjacent_cells=False, include_cells=False)
test_xf = threshed[0].extract_points(xf_multi,adjacent_cells=False, include_cells=False)

# little finger
yd5 = threshed[0].points[spmT_d5_full,1]
xd5 = threshed[0].points[spmT_d5_full,0]
print(yd5)
yd5_min = yd5-(avg_v2v_dist/2)
yd5_max = yd5+(avg_v2v_dist/2)
xd5_min = xd5-(avg_v2v_dist/2)
xd5_max = xd5+(avg_v2v_dist/2)
print(yd5_max)
print(xd5_max)
yd5_multi = np.where((threshed[0].points[:,1]>yd5_min) & (threshed[0].points[:,1]<yd5_max) & (threshed[0]['qT1']>1000))
xd5_multi = np.where((threshed[0].points[:,0]>xd5_min) & (threshed[0].points[:,0]<xd5_max) & (threshed[0]['qT1']>1000))
print(yd5_multi[0])
print(xd5_multi[0])

yd5_multi_yval = np.argsort(threshed[0].points[yd5_multi[0],0])
yd5_multi_yidx = np.take_along_axis(yd5_multi[0],yd5_multi_yval,axis=0)
print(yd5_multi_yval)
print(yd5_multi_yidx)

xd5_multi_yval = np.argsort(threshed[0].points[xd5_multi[0],1])
xd5_multi_yidx = np.take_along_axis(xd5_multi[0],xd5_multi_yval,axis=0)
print(xd5_multi_yval)
print(xd5_multi_yidx)

test_yd5 = threshed[0].extract_points(yd5_multi,adjacent_cells=False, include_cells=False)
test_xd5 = threshed[0].extract_points(xd5_multi,adjacent_cells=False, include_cells=False)

# extract multiple equally-spaced start and end points along y-axis
###################################################################
test_yh_num = int(len(yh_multi[0])/5)
print(test_yh_num)
test_yh_offset = int((int((len(yh_multi[0])-(test_yh_num*5)))+test_yh_num)/2)
print(test_yh_offset)
test_yh_red = threshed[0].extract_points(yh_multi_yidx[test_yh_offset::test_yh_num],adjacent_cells=False, include_cells=False)
yh_multi_red = yh_multi_yidx[test_yh_offset::test_yh_num]
yh_multi_red_sort_keys = np.argsort(threshed[0].points[yh_multi_red,0]) 
yh_multi_red_sort = np.take_along_axis(yh_multi_red,yh_multi_red_sort_keys,axis=0) 
print(yh_multi_red)
print(yh_multi_red_sort)
print(yh_multi_red_sort_keys)

test_yf_num = int(len(yf_multi[0])/5)
print(test_yf_num)
test_yf_offset = int((int((len(yf_multi[0])-(test_yf_num*5)))+test_yf_num)/2)
print(test_yf_offset)
test_yf_red = threshed[0].extract_points(yf_multi_yidx[test_yf_offset::test_yf_num],adjacent_cells=False, include_cells=False)
yf_multi_red = yf_multi_yidx[test_yf_offset::test_yf_num]
yf_multi_red_sort_keys = np.argsort(threshed[0].points[yf_multi_red,0]) 
yf_multi_red_sort = np.take_along_axis(yf_multi_red,yf_multi_red_sort_keys,axis=0) 
print(yf_multi_red)
print(threshed[0].points[yf_multi_red,1])
print(yf_multi_red_sort)
print(yf_multi_red_sort_keys)

test_yd5_num = int(len(yd5_multi[0])/5)
print(test_yd5_num)
test_yd5_offset = int((int((len(yd5_multi[0])-(test_yd5_num*5)))+test_yd5_num)/2)
print(test_yd5_offset)
print()
print()
print()
test_yd5_red = threshed[0].extract_points(yd5_multi_yidx[test_yd5_offset::test_yd5_num],adjacent_cells=False, include_cells=False)
print(test_yd5_red)
print()
print()
print()
yd5_multi_red = yd5_multi_yidx[test_yd5_offset::test_yd5_num]
yd5_multi_red_sort_keys = np.argsort(threshed[0].points[yd5_multi_red,0]) 
yd5_multi_red_sort = np.take_along_axis(yd5_multi_red,yd5_multi_red_sort_keys,axis=0) 
print(yd5_multi_red)
print(threshed[0].points[yd5_multi_red,1])
print(yd5_multi_red_sort)
print(yd5_multi_red_sort_keys)

# extract multiple equally-spaced start and end points along x-axis
###################################################################
test_xh_num = int(len(xh_multi[0])/5)
print(test_xh_num)
test_xh_offset = int((int((len(xh_multi[0])-(test_xh_num*5)))+test_xh_num)/2)
print(test_xh_offset)
test_xh_red = threshed[0].extract_points(xh_multi_yidx[test_xh_offset::test_xh_num],adjacent_cells=False, include_cells=False)
xh_multi_red = xh_multi_yidx[test_xh_offset::test_xh_num]
xh_multi_red_sort_keys = np.argsort(threshed[0].points[xh_multi_red,1]) 
xh_multi_red_sort = np.take_along_axis(xh_multi_red,xh_multi_red_sort_keys,axis=0) 
print(xh_multi_red)
print(xh_multi_red_sort)
print(xh_multi_red_sort_keys)

test_xf_num = int(len(xf_multi[0])/5)
print(test_xf_num)
test_xf_offset = int((int((len(xf_multi[0])-(test_xf_num*5)))+test_xf_num)/2)
print(test_xf_offset)
test_xf_red = threshed[0].extract_points(xf_multi_yidx[test_xf_offset::test_xf_num],adjacent_cells=False, include_cells=False)
xf_multi_red = xf_multi_yidx[test_xf_offset::test_xf_num]
xf_multi_red_sort_keys = np.argsort(threshed[0].points[xf_multi_red,1]) 
xf_multi_red_sort = np.take_along_axis(xf_multi_red,xf_multi_red_sort_keys,axis=0) 
print(xf_multi_red)
print(threshed[0].points[xf_multi_red,1])
print(xf_multi_red_sort)
print(xf_multi_red_sort_keys)

test_xd5_num = int(len(xd5_multi[0])/5)
print(test_xd5_num)
test_xd5_offset = int((int((len(xd5_multi[0])-(test_xd5_num*5)))+test_xd5_num)/2)
print(test_xd5_offset)
test_xd5_red = threshed[0].extract_points(xd5_multi_yidx[test_xd5_offset::test_xd5_num],adjacent_cells=False, include_cells=False)
xd5_multi_red = xd5_multi_yidx[test_xd5_offset::test_xd5_num]
xd5_multi_red_sort_keys = np.argsort(threshed[0].points[xd5_multi_red,1]) 
xd5_multi_red_sort = np.take_along_axis(xd5_multi_red,xd5_multi_red_sort_keys,axis=0) 
print(xd5_multi_red)
print(threshed[0].points[xd5_multi_red,1])
print(xd5_multi_red_sort)
print(xd5_multi_red_sort_keys)


In [ ]:
# calculate shortest paths between extracted start and end points along y-axis
##############################################################################
# face-D1 paths
i = 0
path_f_h_full_multi_yred = []
distance_f_h_multi_yred = []
qT1_dp_path_f_h_full_multi_yred = []

for i in range(len(yh_multi_red)):
    path_f_h_full_multi_yred.append(threshed[0].geodesic(yf_multi_red_sort[i], yh_multi_red_sort[i]))
    distance_f_h_multi_yred.append(threshed[0].geodesic_distance(yf_multi_red_sort[i], yh_multi_red_sort[i]))
    qT1_dp_path_f_h_full_multi_yred.append(threshed[0]['qT1'][path_f_h_full_multi_yred[i]['vtkOriginalPointIds']])

In [ ]:
# D2-D5 paths
i = 0
path_d2_d5_full_multi_yred = []
distance_d2_d5_multi_yred = []
qT1_dp_path_d2_d5_full_multi_yred = []

for i in range(len(yh_multi_red)):
    path_d2_d5_full_multi_yred.append(threshed[0].geodesic(yh_multi_red_sort[i], yd5_multi_red_sort[i]))
    distance_d2_d5_multi_yred.append(threshed[0].geodesic_distance(yh_multi_red_sort[i], yd5_multi_red_sort[i]))
    qT1_dp_path_d2_d5_full_multi_yred.append(threshed[0]['qT1'][path_d2_d5_full_multi_yred[i]['vtkOriginalPointIds']])    

In [ ]:
# calculate shortest paths between extracted start and end points along x-axis
##############################################################################
# face-D1 paths

i = 0
path_f_h_full_multi_xred = []
distance_f_h_multi_xred = []
qT1_dp_path_f_h_full_multi_xred = []

for i in range(len(xh_multi_red)):
    path_f_h_full_multi_xred.append(threshed[0].geodesic(xf_multi_red_sort[i], xh_multi_red_sort[i]))
    distance_f_h_multi_xred.append(threshed[0].geodesic_distance(xf_multi_red_sort[i], xh_multi_red_sort[i]))
    qT1_dp_path_f_h_full_multi_xred.append(threshed[0]['qT1'][path_f_h_full_multi_xred[i]['vtkOriginalPointIds']])
    
# D5-D5 paths
i = 0
path_d2_d5_full_multi_xred = []
distance_d2_d5_multi_xred = []
qT1_dp_path_d2_d5_full_multi_xred = []

for i in range(len(xh_multi_red)):
    path_d2_d5_full_multi_xred.append(threshed[0].geodesic(xh_multi_red_sort[i], xd5_multi_red_sort[i]))
    distance_d2_d5_multi_xred.append(threshed[0].geodesic_distance(xh_multi_red_sort[i], xd5_multi_red_sort[i]))
    qT1_dp_path_d2_d5_full_multi_xred.append(threshed[0]['qT1'][path_d2_d5_full_multi_xred[i]['vtkOriginalPointIds']])    

In [ ]:
# project extracted original geod. path to surface and visualize #
##################################################################
vtx_mesh_qT1_dp = mesh_qT1_dp['EmbedVertex']
vtx_mesh_qT1_dp_omitzero = vtx_mesh_qT1_dp[np.nonzero(vtx_mesh_qT1_dp)]
mean_dp = vtx_mesh_qT1_dp_omitzero.mean()
sd_dp = vtx_mesh_qT1_dp_omitzero.std()

print(mean_dp)
print(sd_dp)

p4 = pv.Plotter(off_screen=True)
#p4.add_mesh(mesh_qT1_dp, cmap=qT1_cmap, clim=[mean_dp-(4*sd_dp), mean_dp+(4*sd_dp)],below_color="#ffffff")
p4.add_mesh(mesh_f, cmap='RdPu')
p4.add_mesh(path_d1_dist10, line_width=8, color='k')
p4.add_mesh(path_d2_d5_full, line_width=8, color='k')
p4.add_mesh(extracted_fd10, point_size=8, color='c')
#p4.add_mesh(threshed[0].extract_points(zf_multi[0],adjacent_cells=False, include_cells=False))
#p4.add_mesh(threshed[0].extract_points(zd5_multi[0],adjacent_cells=False, include_cells=False))
p4.add_mesh(extracted_h, point_size=8, color='r')
p4.add_mesh(extracted_d5, point_size=8, color='y')
#p4.add_mesh(extracted_d5_old, point_size=8, color='w')
p4.camera_position = p4_camera_position
p4.show()

p4 = pv.Plotter(off_screen=True)
p4.add_mesh(mesh_qT1_dp, cmap=qT1_cmap, clim=[mean_dp-(4*sd_dp), mean_dp+(4*sd_dp)],below_color="#ffffff")
p4.add_mesh(path_d1_dist10, line_width=8, color='k')
p4.add_mesh(path_d2_d5_full, line_width=8, color='k')
p4.add_mesh(extracted_fd10, point_size=8, color='c')
p4.add_mesh(extracted_h, point_size=8, color='r')
p4.add_mesh(extracted_d5, point_size=8, color='y')
p4.camera_position = p4_camera_position
p4.screenshot(outdir+ind+'_qT1_dp_paths.png',transparent_background = True)

In [ ]:
# extract geodesic distances for all paths along y-axis
#################################################################################################
distance_f_h_all_multi_y = [[0],[0],[0],[0],[0]]
distance_f_h_all_multi_y_rev = [[0],[0],[0],[0],[0]]
i = 0
j = 0

for i in range(len(path_f_h_full_multi_yred)):
    for j in range(len(path_f_h_full_multi_yred[i]['vtkOriginalPointIds'])-1):
        cur_dist = threshed[0].geodesic_distance(path_f_h_full_multi_yred[i]['vtkOriginalPointIds'][0], path_f_h_full_multi_yred[i]['vtkOriginalPointIds'][j+1])
        distance_f_h_all_multi_y[i].append(cur_dist)
    
    distance_f_h_all_multi_y_rev[i] = np.array(distance_f_h_all_multi_y[i]) - np.max(distance_f_h_all_multi_y[i])

#print(distance_f_h_all_multi_y)
print(distance_f_h_all_multi_y_rev)


distance_d2_d5_all_multi_y = [[0],[0],[0],[0],[0]]
distance_d2_d5_all_multi_y_fin = [[0],[0],[0],[0],[0]]
i = 0
j = 0

for i in range(len(path_d2_d5_full_multi_yred)):
    for j in range(len(path_d2_d5_full_multi_yred[i]['vtkOriginalPointIds'])-1):
        cur_dist = threshed[0].geodesic_distance(path_d2_d5_full_multi_yred[i]['vtkOriginalPointIds'][0], path_d2_d5_full_multi_yred[i]['vtkOriginalPointIds'][j+1])
        distance_d2_d5_all_multi_y[i].append(cur_dist)
        
    distance_d2_d5_all_multi_y_fin[i] = np.array(distance_d2_d5_all_multi_y[i])
    
#print(distance_d2_d5_all_multi_y)
print(distance_d2_d5_all_multi_y_fin)

In [ ]:
# extract geodesic distances for all paths sampled along x-axis
#################################################################################################
distance_f_h_all_multi_x = [[0],[0],[0],[0],[0]]
distance_f_h_all_multi_x_rev = [[0],[0],[0],[0],[0]]

i = 0
j = 0

for i in range(len(path_f_h_full_multi_xred)):
    for j in range(len(path_f_h_full_multi_xred[i]['vtkOriginalPointIds'])-1):
        cur_dist = threshed[0].geodesic_distance(path_f_h_full_multi_xred[i]['vtkOriginalPointIds'][0], path_f_h_full_multi_xred[i]['vtkOriginalPointIds'][j+1])
        distance_f_h_all_multi_x[i].append(cur_dist)
    
    distance_f_h_all_multi_x_rev[i] = np.array(distance_f_h_all_multi_x[i]) - np.max(distance_f_h_all_multi_x[i])

#print(distance_f_h_all_multi_x)
print(distance_f_h_all_multi_x_rev)

distance_d2_d5_all_multi_x = [[0],[0],[0],[0],[0]]
distance_d2_d5_all_multi_x_fin = [[0],[0],[0],[0],[0]]
i = 0
j = 0
for i in range(len(path_d2_d5_full_multi_xred)):
    for j in range(len(path_d2_d5_full_multi_xred[i]['vtkOriginalPointIds'])-1):
        cur_dist = threshed[0].geodesic_distance(path_d2_d5_full_multi_xred[i]['vtkOriginalPointIds'][0], path_d2_d5_full_multi_xred[i]['vtkOriginalPointIds'][j+1])
        distance_d2_d5_all_multi_x[i].append(cur_dist)
        
    distance_d2_d5_all_multi_x_fin[i] = np.array(distance_d2_d5_all_multi_x[i])
    
#print(distance_d2_d5_all_multi_x)
print(distance_d2_d5_all_multi_x_fin)

In [ ]:
# plot vertices along y-axis
############################
# deep qT1
p4 = pv.Plotter()
p4.add_mesh(mesh_qT1_dp, cmap=qT1_cmap, clim=[mean_dp-(4*sd_dp), mean_dp+(4*sd_dp)],below_color="#ffffff")
p4.add_mesh(test_yd5)
p4.add_mesh(test_yh)
p4.add_mesh(test_yf)
p4.camera_position = p4_camera_position
p4.show()

In [ ]:
#plot reduced vertices and geodesic paths along y-axis
######################################################
# deep qT1
i=0
p4 = pv.Plotter()
#p4.add_mesh(mesh_qT1_dp, cmap=qT1_cmap, clim=[mean_dp-(4*sd_dp), mean_dp+(4*sd_dp)],below_color="#ffffff")
p4.add_mesh(mesh_f, cmap='RdPu')
p4.add_mesh(test_yh_red, point_size=8, color='r')
p4.add_mesh(test_yf_red,point_size=8, color='c')
p4.add_mesh(test_yd5_red,point_size=8, color='y')
#for i in range(len(path_f_h_full_multi_yred)):
    #p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
    #p4.add_mesh(path_h_d2_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.show()

p4 = pv.Plotter(off_screen=True)
p4.add_mesh(mesh_qT1_dp, cmap=qT1_cmap, clim=[mean_dp-(4*sd_dp), mean_dp+(4*sd_dp)],below_color="#ffffff")
p4.add_mesh(test_yh_red, point_size=8, color='r')
p4.add_mesh(test_yf_red,point_size=8, color='c')
p4.add_mesh(test_yd5_red,point_size=8, color='y')
#for i in range(len(path_f_h_full_multi_yred)):
    #p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
    #p4.add_mesh(path_h_d2_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.screenshot(outdir+ind+'_multi_points_y_deep.png',transparent_background=True)

In [ ]:
#plot reduced vertices and geodesic paths along y-axis
######################################################
# deep qT1
i=0
p4 = pv.Plotter()
p4.add_mesh(mesh_qT1_dp, cmap=qT1_cmap, clim=[mean_dp-(4*sd_dp), mean_dp+(4*sd_dp)],below_color="#ffffff")
p4.add_mesh(test_yh_red, point_size=8, color='r')
p4.add_mesh(test_yf_red,point_size=8, color='c')
p4.add_mesh(test_yd5_red,point_size=8, color='y')
for i in range(len(path_f_h_full_multi_yred)):
    p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
    p4.add_mesh(path_d2_d5_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.show()

p4 = pv.Plotter(off_screen=True)
p4.add_mesh(mesh_qT1_dp, cmap=qT1_cmap, clim=[mean_dp-(4*sd_dp), mean_dp+(4*sd_dp)],below_color="#ffffff")
p4.add_mesh(test_yh_red, point_size=8, color='r')
p4.add_mesh(test_yf_red,point_size=8, color='c')
p4.add_mesh(test_yd5_red,point_size=8, color='y')
for i in range(len(path_f_h_full_multi_yred)):
    p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
    p4.add_mesh(path_d2_d5_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.screenshot(outdir+ind+'_multi_paths_y_deep.png',transparent_background=True)

In [ ]:
# plot vertices along x-axis
############################
# deep qT1
p4 = pv.Plotter()
p4.add_mesh(mesh_qT1_dp, cmap=qT1_cmap, clim=[mean_dp-(4*sd_dp), mean_dp+(4*sd_dp)],below_color="#ffffff")
p4.add_mesh(test_xd5)
p4.add_mesh(test_xh)
p4.add_mesh(test_xf)
p4.camera_position = p4_camera_position
p4.show()

In [ ]:
#plot reduced vertices and geodesic paths along x-axis
######################################################
# deep qT1
i=0
p4 = pv.Plotter()
#p4.add_mesh(mesh_qT1_dp, cmap=qT1_cmap, clim=[mean_dp-(4*sd_dp), mean_dp+(4*sd_dp)],below_color="#ffffff")
p4.add_mesh(mesh_f, cmap='RdPu')
p4.add_mesh(test_xh_red, point_size=8, color='r')
p4.add_mesh(test_xf_red,point_size=8, color='c')
p4.add_mesh(test_xd5_red,point_size=8, color='y')
#for i in range(len(path_f_h_full_multi_yred)):
#    p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.show()

p4 = pv.Plotter(off_screen=True)
p4.add_mesh(mesh_qT1_dp, cmap=qT1_cmap, clim=[mean_dp-(3.5*sd_dp), mean_dp+(3.5*sd_dp)],below_color="#ffffff")
p4.add_mesh(test_xh_red, point_size=8, color='r')
p4.add_mesh(test_xf_red,point_size=8, color='c')
p4.add_mesh(test_xd5_red,point_size=8, color='y')
#for i in range(len(path_f_h_full_multi_yred)):
#    p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.screenshot(outdir+ind+'_multi_points_x_deep.png',transparent_background=True)

In [ ]:
#plot reduced vertices and geodesic paths along x-axis
######################################################
# deep qT1
i=0
p4 = pv.Plotter()
p4.add_mesh(mesh_qT1_dp, cmap=qT1_cmap, clim=[mean_dp-(4*sd_dp), mean_dp+(4*sd_dp)],below_color="#ffffff")
p4.add_mesh(test_xh_red, point_size=8, color='r')
p4.add_mesh(test_xf_red,point_size=8, color='c')
p4.add_mesh(test_xd5_red,point_size=8, color='y')
for i in range(len(path_f_h_full_multi_xred)):
    p4.add_mesh(path_f_h_full_multi_xred[i], line_width=4, color='k')
    p4.add_mesh(path_d2_d5_full_multi_xred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.show()

p4 = pv.Plotter(off_screen=True)
p4.add_mesh(mesh_qT1_dp, cmap=qT1_cmap, clim=[mean_dp-(4*sd_dp), mean_dp+(4*sd_dp)],below_color="#ffffff")
p4.add_mesh(test_xh_red, point_size=8, color='r')
p4.add_mesh(test_xf_red,point_size=8, color='c')
p4.add_mesh(test_xd5_red,point_size=8, color='y')
for i in range(len(path_f_h_full_multi_xred)):
    p4.add_mesh(path_f_h_full_multi_xred[i], line_width=4, color='k')
    p4.add_mesh(path_d2_d5_full_multi_xred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.screenshot(outdir+ind+'_multi_paths_x_deep.png',transparent_background=True)

In [ ]:
# plot path
# middle qT1
mesh_qT1_im = pv.read(folder_qT1+filename_qT1_im)
vtx_mesh_qT1_im = mesh_qT1_im['EmbedVertex']
vtx_mesh_qT1_im_omitzero = vtx_mesh_qT1_im[np.nonzero(vtx_mesh_qT1_im)]
mean_im = vtx_mesh_qT1_im_omitzero.mean()
sd_im = vtx_mesh_qT1_im_omitzero.std()

p4 = pv.Plotter(off_screen=True)
p4.add_mesh(mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p4.add_mesh(path_d1_dist10, line_width=8, color='k')
p4.add_mesh(path_d2_d5_full, line_width=8, color='k')
p4.add_mesh(extracted_fd10, point_size=8, color='c')
p4.add_mesh(extracted_d5, point_size=8, color='y')
p4.add_mesh(extracted_h, point_size=8, color='r')
p4.camera_position = p4_camera_position
p4.show()

p4 = pv.Plotter(off_screen=True)
p4.add_mesh(mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p4.add_mesh(path_d1_dist10, line_width=8, color='k')
p4.add_mesh(path_d2_d5_full, line_width=8, color='k')
p4.add_mesh(extracted_fd10, point_size=8, color='c')
p4.add_mesh(extracted_d5, point_size=8, color='y')
p4.add_mesh(extracted_h, point_size=8, color='r')
p4.camera_position = p4_camera_position
p4.screenshot(outdir+ind+'_qT1_im_paths.png',transparent_background = True)
#p4.savefig('test')

In [ ]:
#plot reduced vertices and geodesic paths along y-axis
######################################################
# middle qT1
i=0
p4 = pv.Plotter()
p4.add_mesh(mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p4.add_mesh(test_yh_red, point_size=6, color='r')
p4.add_mesh(test_yf_red,point_size=6, color='c')
p4.add_mesh(test_yd5_red,point_size=6, color='y')
#for i in range(len(path_f_h_full_multi_yred)):
    #p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
    #p4.add_mesh(path_h_d2_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.show()

p4 = pv.Plotter(off_screen=True)
p4.add_mesh(mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p4.add_mesh(test_yh_red, point_size=6, color='r')
p4.add_mesh(test_yf_red,point_size=6, color='c')
p4.add_mesh(test_yd5_red,point_size=6, color='y')
#for i in range(len(path_f_h_full_multi_yred)):
    #p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
    #p4.add_mesh(path_h_d2_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.screenshot(outdir+ind+'_multi_points_y_middle.png',transparent_background=True)

In [ ]:
#plot reduced vertices and geodesic paths along x-axis
######################################################
# middle qT1
i=0
p4 = pv.Plotter()
p4.add_mesh(mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p4.add_mesh(test_xh_red, point_size=6, color='r')
p4.add_mesh(test_xf_red,point_size=6, color='c')
p4.add_mesh(test_xd5_red,point_size=6, color='y')
#for i in range(len(path_f_h_full_multi_yred)):
#    p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.show()

p4 = pv.Plotter(off_screen=True)
p4.add_mesh(mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p4.add_mesh(test_xh_red, point_size=6, color='r')
p4.add_mesh(test_xf_red,point_size=6, color='c')
p4.add_mesh(test_xd5_red,point_size=6, color='y')
#for i in range(len(path_f_h_full_multi_yred)):
#    p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.screenshot(outdir+ind+'_multi_points_x_middle.png',transparent_background=True)

In [ ]:
path_d2_d1_d10_multi_y = []

for j in range (len(path_f_h_full_multi_yred)):
    path_d2_d1_d10_multi_y.append(np.concatenate((path_f_h_full_multi_yred[j]['vtkOriginalPointIds'],path_d2_d5_full_multi_yred[j]['vtkOriginalPointIds'][1:])))
    print(path_f_h_full_multi_yred[j]['vtkOriginalPointIds'])
print(path_d2_d1_d10_multi_y)

In [ ]:
# extract layer-specific qT1 along geodesic path
#################################################
qT1_dp_path_f_h_full = threshed[0]['qT1'][path_f_h_full['vtkOriginalPointIds']]
qT1_dp_path_d2_d5_full = threshed[0]['qT1'][path_d2_d5_full['vtkOriginalPointIds']]
print(qT1_dp_path_f_h_full)

qT1_im_path_f_h_full = threshed[0]['middleqT1'][path_f_h_full['vtkOriginalPointIds']]
qT1_im_path_d2_d5_full = threshed[0]['middleqT1'][path_d2_d5_full['vtkOriginalPointIds']]
print(qT1_im_path_f_h_full)

qT1_sim_path_f_h_full = threshed[0]['smoothmiddleqT1'][path_f_h_full['vtkOriginalPointIds']]
qT1_sim_path_d2_d5_full = threshed[0]['smoothmiddleqT1'][path_d2_d5_full['vtkOriginalPointIds']]
print(qT1_sim_path_f_h_full)

# concatenate extracted qT1 values and geodesic distances from different geodesic paths
#######################################################################################
qT1_dp_path_d2_d1_d10 = np.concatenate((qT1_dp_path_f_h_full, qT1_dp_path_d2_d5_full))
qT1_im_path_d2_d1_d10 = np.concatenate((qT1_im_path_f_h_full, qT1_im_path_d2_d5_full))
qT1_sim_path_d2_d1_d10 = np.concatenate((qT1_sim_path_f_h_full, qT1_sim_path_d2_d5_full))
distance_f_h_all_d2_d1_d10 = np.concatenate((distance_f_h_all_rev_d10,distance_d2_d5_all_fin))
#print(qT1_im_path_d2_d1_d10)
print(len(qT1_im_path_d2_d1_d10))
print(len(qT1_im_path_f_h_full))
print(len(distance_f_h_all_rev_d10))
print(len(distance_f_h_all_d2_d1_d10))
print(qT1_sim_path_d2_d1_d10)

In [ ]:
# extract layer-specific qT1 values from multiple paths along y-axis
####################################################################
i=0
qT1_im_path_f_h_full_multi_y = []
qT1_sim_path_f_h_full_multi_y = []

qT1_im_path_h_d2_full_multi_y = []
qT1_sim_path_h_d2_full_multi_y = []

qT1_im_path_d2_d5_full_multi_y = []
qT1_sim_path_d2_d5_full_multi_y = []

for i in range(len(path_f_h_full_multi_yred)):
    qT1_im_path_f_h_full_multi_y.append(threshed[0]['middleqT1'][path_f_h_full_multi_yred[i]['vtkOriginalPointIds']])
    qT1_sim_path_f_h_full_multi_y.append(threshed[0]['smoothmiddleqT1'][path_f_h_full_multi_yred[i]['vtkOriginalPointIds']])
       
print(path_f_h_full_multi_yred)

for i in range(len(path_d2_d5_full_multi_yred)):
    qT1_im_path_d2_d5_full_multi_y.append(threshed[0]['middleqT1'][path_d2_d5_full_multi_yred[i]['vtkOriginalPointIds']])
    qT1_sim_path_d2_d5_full_multi_y.append(threshed[0]['smoothmiddleqT1'][path_d2_d5_full_multi_yred[i]['vtkOriginalPointIds']])
    
    
# extract layer-specific qT1 values from multiple paths along x-axis
####################################################################
i=0
qT1_im_path_f_h_full_multi_x = []
qT1_sim_path_f_h_full_multi_x = []

qT1_im_path_h_d2_full_multi_x = []
qT1_sim_path_h_d2_full_multi_x = []

qT1_im_path_d2_d5_full_multi_x = []
qT1_sim_path_d2_d5_full_multi_x = []

for i in range(len(path_f_h_full_multi_xred)):
    qT1_im_path_f_h_full_multi_x.append(threshed[0]['middleqT1'][path_f_h_full_multi_xred[i]['vtkOriginalPointIds']])
    qT1_sim_path_f_h_full_multi_x.append(threshed[0]['smoothmiddleqT1'][path_f_h_full_multi_xred[i]['vtkOriginalPointIds']])
    
for i in range(len(path_d2_d5_full_multi_xred)):
    qT1_im_path_d2_d5_full_multi_x.append(threshed[0]['middleqT1'][path_d2_d5_full_multi_xred[i]['vtkOriginalPointIds']])
    qT1_sim_path_d2_d5_full_multi_x.append(threshed[0]['smoothmiddleqT1'][path_d2_d5_full_multi_xred[i]['vtkOriginalPointIds']])
    

In [ ]:
# concatenate extracted qT1 values and geodesic distances from different geodesic paths for all sampled paths along y-axis
##########################################################################################################################
j=0
qT1_im_path_d2_d1_d10_multi_y = []
qT1_sim_path_d2_d1_d10_multi_y = []
distance_f_h_all_d2_d1_d10_multi_y = []

for j in range (len(path_f_h_full_multi_yred)):
    qT1_im_path_d2_d1_d10_multi_y.append(np.concatenate((qT1_im_path_f_h_full_multi_y[j], qT1_im_path_d2_d5_full_multi_y[j][1:])))
    qT1_sim_path_d2_d1_d10_multi_y.append(np.concatenate((qT1_sim_path_f_h_full_multi_y[j], qT1_sim_path_d2_d5_full_multi_y[j][1:])))
    distance_f_h_all_d2_d1_d10_multi_y.append(np.concatenate((distance_f_h_all_multi_y_rev[j],distance_d2_d5_all_multi_y_fin[j][1:])))

print(qT1_im_path_d2_d1_d10_multi_y)
print(distance_f_h_all_d2_d1_d10_multi_y)

In [ ]:
# concatenate extracted qT1 values and geodesic distances from different geodesic paths for all sampled paths along x-axis
##########################################################################################################################
j=0
qT1_im_path_d2_d1_d10_multi_x = []
qT1_sim_path_d2_d1_d10_multi_x = []
distance_f_h_all_d2_d1_d10_multi_x = []

for j in range (len(path_f_h_full_multi_xred)):
    qT1_im_path_d2_d1_d10_multi_x.append(np.concatenate((qT1_im_path_f_h_full_multi_x[j], qT1_im_path_d2_d5_full_multi_x[j][1:])))
    qT1_sim_path_d2_d1_d10_multi_x.append(np.concatenate((qT1_sim_path_f_h_full_multi_x[j], qT1_sim_path_d2_d5_full_multi_x[j][1:])))
    distance_f_h_all_d2_d1_d10_multi_x.append(np.concatenate((distance_f_h_all_multi_x_rev[j], distance_d2_d5_all_multi_x_fin[j][1:])))

print(qT1_im_path_d2_d1_d10_multi_x)
print(distance_f_h_all_d2_d1_d10_multi_x)

In [ ]:
# 3D plots of multiple paths along y-axis
#########################################

from mpl_toolkits.mplot3d.art3d import Line3DCollection

# set up a figure twice as wide as it is tall
fig = plt.figure(figsize=plt.figaspect(8))

# =============
# Second subplot
# =============
# set up the axes for the first plot
ax = fig.add_subplot(3, 1, 2, projection='3d', computed_zorder=False)

i = 0

yrange = [0,1,2,3,4]

for k in reversed(yrange):
    
    yvalues = [k]*len(distance_f_h_all_d2_d1_d10_multi_y[k])
    points = np.array([distance_f_h_all_d2_d1_d10_multi_y[k], yvalues, qT1_im_path_d2_d1_d10_multi_y[k]]).T.reshape(-1, 1, 3)
    segments = np.concatenate([points[:-2],points[1:-1], points[2:]], axis=1)
    
    lc = Line3DCollection(segments, cmap='viridis')
    # Set the values used for colormapping
    lc.set_array(qT1_im_path_d2_d1_d10_multi_y[k])
    lc.set_linewidth(4)
    
    ax.add_collection3d(lc)
    ax.set_title('middle')
    ax.set(zlabel='qT1 [ms]')
    ax.set(xlabel='geodesic distance [mm]')
    ax.set(ylabel='path number')
    ax.set_ylim(min(yrange),max(yrange))
    ax.set_zlim(2800,1200)
    ax.set_zticks([2800,1200])
    ax.set_yticks([0,4])
    ax.set_xticks([min(distance_f_h_all_d2_d1_d10_multi_y[k]),0,max(distance_f_h_all_d2_d1_d10_multi_y[k])])
    ax.set_xlim(min(distance_f_h_all_d2_d1_d10_multi_y[k]),max(distance_f_h_all_d2_d1_d10_multi_y[k]))
    
    ax.plot(distance_f_h_all_d2_d1_d10_multi_y[k][np.where(distance_f_h_all_d2_d1_d10_multi_y[k]==0)[0][0]], k, qT1_im_path_d2_d1_d10_multi_y[k][np.where(distance_f_h_all_d2_d1_d10_multi_y[k]==0)[0][0]], color='r', marker='.', markersize=12)


fig.tight_layout()

In [ ]:
# save plots
############
fig.savefig(outdir+ind+'_3D_multi_path_mulicolor_y.png')
fig.savefig(outdir+ind+'_3D_multi_path_mulicolor_y.svg')

In [ ]:
# 3D plots of multiple paths along x-axis
#########################################

from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import PolyCollection

# set up a figure twice as wide as it is tall
fig = plt.figure(figsize=plt.figaspect(8))

# =============
# Second subplot
# =============
# set up the axes for the first plot
ax = fig.add_subplot(3, 1, 2, projection='3d', computed_zorder=False)

i = 0

yrange = [0,1,2,3,4]

for k in reversed(yrange):
    
    yvalues = [k]*len(distance_f_h_all_d2_d1_d10_multi_x[k])
    points = np.array([distance_f_h_all_d2_d1_d10_multi_x[k], yvalues, qT1_im_path_d2_d1_d10_multi_x[k]]).T.reshape(-1, 1, 3)
    segments = np.concatenate([points[:-2],points[1:-1], points[2:]], axis=1)
    
    lc = Line3DCollection(segments, cmap='viridis')
    # Set the values used for colormapping
    lc.set_array(qT1_im_path_d2_d1_d10_multi_x[k])
    lc.set_linewidth(4)
    
    ax.add_collection3d(lc)
    ax.set_title('middle')
    ax.set(zlabel='qT1 [ms]')
    ax.set(xlabel='geodesic distance [mm]')
    ax.set(ylabel='path number')
    ax.set_ylim(min(yrange),max(yrange))
    ax.set_zlim(2800,1200)
    ax.set_zticks([2800,1200])
    ax.set_yticks(yrange)
    ax.set_xticks([min(distance_f_h_all_d2_d1_d10_multi_x[k]),0,max(distance_f_h_all_d2_d1_d10_multi_x[k])])
    ax.set_xlim(min(distance_f_h_all_d2_d1_d10_multi_x[k]),max(distance_f_h_all_d2_d1_d10_multi_x[k]))
    
    ax.plot(distance_f_h_all_d2_d1_d10_multi_x[k][np.where(distance_f_h_all_d2_d1_d10_multi_x[k]==0)[0][0]], k, qT1_im_path_d2_d1_d10_multi_x[k][np.where(distance_f_h_all_d2_d1_d10_multi_x[k]==0)[0][0]], color='r', marker='.', markersize=12)

fig.tight_layout()

In [ ]:
# save plots
############
fig.savefig(outdir+ind+'_3D_multi_path_mulicolor_x.png')
fig.savefig(outdir+ind+'_3D_multi_path_mulicolor_x.svg')

In [ ]:
# publication-ready 3D plots of multiple paths for middle along x-axis
######################################################################

from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import PolyCollection
from matplotlib import rcParams

rcParams['axes.labelpad'] = 25

# set up a figure twice as wide as it is tall
fig = plt.figure(figsize=plt.figaspect(8))

# =============
# First subplot
# =============
# set up the axes for the first plot
ax = fig.add_subplot(3, 1, 1, projection='3d', computed_zorder=False)

i = 0

yrange = [0,1,2,3,4]
verts = []

for k in yrange:
    
    yvalues = [k]*len(distance_f_h_all_d2_d1_d10_multi_x[k])
    points = np.array([distance_f_h_all_d2_d1_d10_multi_x[k], yvalues, qT1_im_path_d2_d1_d10_multi_x[k]]).T.reshape(-1, 1, 3)
    segments = np.concatenate([points[:-2],points[1:-1], points[2:]], axis=1)
    
    lc = Line3DCollection(segments, cmap='viridis')
    # Set the values used for colormapping
    lc.set_array(qT1_im_path_d2_d1_d10_multi_x[k])
    lc.set_linewidth(4)
    
    ax.add_collection3d(lc)
    #ax.set_title('middle')
    #ax.set(zlabel='qT1 [ms]')
    #ax.set(xlabel='geodesic distance\n[mm]')
    #ax.set(ylabel='path number')
    ax.set_xlabel('geodesic distance [mm]\ninferior > superior',labelpad=30, fontsize=20)
    ax.set_ylabel('path number\nanterior > posterior',labelpad=30, fontsize=20)
    ax.set_zlabel('qT1\n[ms]',labelpad=30, fontsize=20)
    ax.tick_params(axis='both', labelsize=20,pad=5,width=25)
    ax.tick_params(axis='z', labelsize=20,pad=15)
    ax.set_ylim(max(yrange),min(yrange))
    ax.set_zlim(2800,1200)
    ax.set_zticks([2800,1200])
    ax.set_yticks([4,0])
    ax.set_xticks([round(min(distance_f_h_all_d2_d1_d10_multi_x[k])-0.5,1),0,round(max(distance_f_h_all_d2_d1_d10_multi_x[k])+0.5,1)])
    ax.set_xlim(round(min(distance_f_h_all_d2_d1_d10_multi_x[k])-0.5,1),round(max(distance_f_h_all_d2_d1_d10_multi_x[k])+0.5,1))
    
    ax.plot(distance_f_h_all_d2_d1_d10_multi_x[k][np.where(distance_f_h_all_d2_d1_d10_multi_x[k]==0)[0][0]], k, qT1_im_path_d2_d1_d10_multi_x[k][np.where(distance_f_h_all_d2_d1_d10_multi_x[k]==0)[0][0]], color='r', marker='.', markersize=14)

fig.tight_layout()

In [ ]:
# save plots
############
fig.savefig(outdir+ind+'_3D_multi_path_mulicolor_x_middle.png')
fig.savefig(outdir+ind+'_3D_multi_path_mulicolor_x_middle.svg')

In [ ]:
# publication-ready 3D plots of multiple paths for middle along y-axis
######################################################################

from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import PolyCollection
from matplotlib import rcParams

rcParams['axes.labelpad'] = 25

# set up a figure twice as wide as it is tall
fig = plt.figure(figsize=plt.figaspect(8))

# =============
# First subplot
# =============
# set up the axes for the first plot
ax = fig.add_subplot(3, 1, 1, projection='3d', computed_zorder=False)

i = 0

yrange = [0,1,2,3,4]
verts = []

for k in reversed(yrange):
    
    yvalues = [k]*len(distance_f_h_all_d2_d1_d10_multi_y[k])
    points = np.array([distance_f_h_all_d2_d1_d10_multi_y[k], yvalues, qT1_im_path_d2_d1_d10_multi_y[k]]).T.reshape(-1, 1, 3)
    segments = np.concatenate([points[:-2],points[1:-1], points[2:]], axis=1)
    
    lc = Line3DCollection(segments, cmap='viridis')
    # Set the values used for colormapping
    lc.set_array(qT1_im_path_d2_d1_d10_multi_y[k])
    lc.set_linewidth(4)
    lc.set_clim(vmin=min(qT1_im_path_d2_d1_d10_multi_y[k]), vmax=max(qT1_im_path_d2_d1_d10_multi_y[k]))
                
    ax.add_collection3d(lc)
    #ax.set_title('middle')
    #ax.set(zlabel='qT1 [ms]')
    #ax.set(xlabel='geodesic distance\n[mm]')
    #ax.set(ylabel='path number')
    ax.set_xlabel('geodesic distance [mm]\ninferior > superior',labelpad=30, fontsize=20)
    ax.set_ylabel('path number\nanterior > posterior',labelpad=30, fontsize=20)
    ax.set_zlabel('qT1\n[ms]',labelpad=30, fontsize=20)
    ax.tick_params(axis='both', labelsize=20,pad=5,width=25)
    ax.tick_params(axis='z', labelsize=20,pad=15)
    ax.set_ylim(min(yrange),max(yrange))
    ax.set_zlim(2800,1200)
    ax.set_zticks([2800,1200])
    ax.set_yticks([0,4])
    ax.set_xticks([round(min(distance_f_h_all_d2_d1_d10_multi_y[k])-0.5,1),0,round(max(distance_f_h_all_d2_d1_d10_multi_y[k])+0.5,1)])
    ax.set_xlim(round(min(distance_f_h_all_d2_d1_d10_multi_y[k])-0.5,1),round(max(distance_f_h_all_d2_d1_d10_multi_y[k])+0.5,1))
    
    ax.plot(distance_f_h_all_d2_d1_d10_multi_y[k][np.where(distance_f_h_all_d2_d1_d10_multi_y[k]==0)[0][0]], k, qT1_im_path_d2_d1_d10_multi_y[k][np.where(distance_f_h_all_d2_d1_d10_multi_y[k]==0)[0][0]], color='r', marker='.', markersize=14)
#fig.colorbar(lc)
fig.tight_layout()

In [ ]:
# save plots
############
fig.savefig(outdir+ind+'_3D_multi_path_mulicolor_y_middle.png')
fig.savefig(outdir+ind+'_3D_multi_path_mulicolor_y_middle.svg')

In [ ]:
#find most prominent peaks on geodesic paths sampled between forehead and little finger
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import PolyCollection
from matplotlib import rcParams

from scipy.signal import find_peaks, peak_prominences

yvalues = []
peaks_m = [[0],[0],[0],[0],[0]]
properties_m = [[0],[0],[0],[0],[0]]
prominences = [[0],[0],[0],[0],[0]]
prominences_prop = [[0],[0],[0],[0],[0]]
max_prominences = [[0],[0],[0],[0],[0]]
maxidx_prominences = [[0],[0],[0],[0],[0]]

#peaks_m[k][maxidx_prominences[k]]

rcParams['axes.labelpad'] = 25

# set up a figure twice as wide as it is tall
fig, axs = plt.subplots(5, 1, sharex=True)

i = 0

yrange = [0,1,2,3,4]

for k in reversed(yrange):
    print(k)
    yvalues.append([k]*len(distance_f_h_all_d2_d1_d10_multi_y[k]))
    peaks_m[k], properties_m[k] = find_peaks(qT1_im_path_d2_d1_d10_multi_y[k],prominence=2*np.std(qT1_im_path_d2_d1_d10_multi_y[k]),width=1)

    axs[k ].plot(distance_f_h_all_d2_d1_d10_multi_y[k],qT1_im_path_d2_d1_d10_multi_y[k])
    axs[k ].plot(distance_f_h_all_d2_d1_d10_multi_y[k][peaks_m[k]], qT1_im_path_d2_d1_d10_multi_y[k][peaks_m[k]], "x")
    axs[k ].vlines(x = distance_f_h_all_d2_d1_d10_multi_y[k][peaks_m[k]], ymin = qT1_im_path_d2_d1_d10_multi_y[k][peaks_m[k]] - properties_m[k]["prominences"], ymax = qT1_im_path_d2_d1_d10_multi_y[k][peaks_m[k]], color = "C1")
    #axs[4-k ].hlines(y = qT1_im_path_d2_d1_d10_multi_y[k][peaks_m[k]] - properties_m[k]["prominences"], xmin = distance_f_h_all_d2_d1_d10_multi_y[k][properties_m[k]["left_bases"]], xmax=distance_f_h_all_d2_d1_d10_multi_y[k][properties_m[k]["right_bases"]], color = "C1")
    #axs[0 ].hlines(*results_full_m[1:], color="C3")
    axs[k ].plot(distance_f_h_all_d2_d1_d10_multi_y[k][0], qT1_im_path_d2_d1_d10_multi_y[k][0], color='c', marker='.', markersize=10)
    axs[k ].plot(distance_f_h_all_d2_d1_d10_multi_y[k][-1], qT1_im_path_d2_d1_d10_multi_y[k][-1], color='y', marker='.', markersize=10)
    axs[k ].plot(0, qT1_im_path_d2_d1_d10_multi_y[k][np.where(distance_f_h_all_d2_d1_d10_multi_y[k]==0)[0][0]], color='r', marker='.', markersize=10)
    #axs[k ].plot(distance_f_h_all_d2_d1_d10_multi_y[k][len(qT1_im_path_f_h_full_multi_y[k])+len(qT1_im_path_h_d2_full_multi_y[k])-1], qT1_im_path_d2_d1_d10_multi_y[k][len(qT1_im_path_f_h_full_multi_y[k])+len(qT1_im_path_h_d2_full_multi_y[k])-1], color='m', marker='.', markersize=10)
    axs[k ].invert_yaxis()
    
axs[k ].set_title('Middle Layer')
    
print(peaks_m)
print(properties_m[k])
print(properties_m[k]["left_ips"])
print(distance_f_h_all_d2_d1_d10_multi_y[k][0])

fig.tight_layout()
#screenshot('/media/doehlerj/WIP-B1/Documents/FINAL/T1_profiles/S1_Paper/results/septa/DDlayers/'+ind+'_face_hand_gradient_myelin_smoothed_peak_prominence_multi_x_'+str(j)+'.png')
fig.savefig(outdir+ind+'_face_hand_myelin_peaks_unsmoothed.png')  

In [ ]:
#plot reduced vertices and geodesic paths along y-axis
######################################################
# smoothed middle qT1
i=0
p4 = pv.Plotter()
p4.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p4.add_mesh(test_yh_red, point_size=6, color='r')
p4.add_mesh(test_yf_red,point_size=6, color='c')
p4.add_mesh(test_yd5_red,point_size=6, color='y')
#for i in range(len(path_f_h_full_multi_yred)):
    #p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
    #p4.add_mesh(path_h_d2_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.show()

p4 = pv.Plotter(off_screen=True)
p4.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p4.add_mesh(test_yh_red, point_size=6, color='r')
p4.add_mesh(test_yf_red,point_size=6, color='c')
p4.add_mesh(test_yd5_red,point_size=6, color='y')
#for i in range(len(path_f_h_full_multi_yred)):
    #p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
    #p4.add_mesh(path_h_d2_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.screenshot(outdir+ind+'_multi_points_y_middle_smooth.png',transparent_background=True)

In [ ]:
print(distance_f_h_all_d2_d1_d10_multi_y)

In [ ]:
#plot reduced vertices and geodesic paths along x-axis
######################################################
# smoothed middle qT1
i=0
p4 = pv.Plotter()
p4.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p4.add_mesh(test_xh_red, point_size=6, color='r')
p4.add_mesh(test_xf_red,point_size=6, color='c')
p4.add_mesh(test_xd5_red,point_size=6, color='y')
#for i in range(len(path_f_h_full_multi_yred)):
    #p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
    #p4.add_mesh(path_h_d2_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.show()

p4 = pv.Plotter(off_screen=True)
p4.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p4.add_mesh(test_xh_red, point_size=6, color='r')
p4.add_mesh(test_xf_red,point_size=6, color='c')
p4.add_mesh(test_xd5_red,point_size=6, color='y')
#for i in range(len(path_f_h_full_multi_yred)):
    #p4.add_mesh(path_f_h_full_multi_yred[i], line_width=4, color='k')
    #p4.add_mesh(path_h_d2_full_multi_yred[i], line_width=4, color='k')
p4.camera_position = p4_camera_position
p4.screenshot(outdir+ind+'_multi_points_x_middle_smooth.png',transparent_background=True)

In [ ]:
# publication-ready 3D plots of multiple paths for middle along y-axis
######################################################################

from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import PolyCollection
from matplotlib import rcParams

rcParams['axes.labelpad'] = 25

# set up a figure twice as wide as it is tall
fig = plt.figure(figsize=plt.figaspect(8))

# =============
# First subplot
# =============
# set up the axes for the first plot
ax = fig.add_subplot(3, 1, 1, projection='3d', computed_zorder=False)

i = 0

yrange = [0,1,2,3,4]
verts = []

for k in reversed(yrange):
    
    yvalues = [k]*len(distance_f_h_all_d2_d1_d10_multi_y[k])
    points = np.array([distance_f_h_all_d2_d1_d10_multi_y[k], yvalues, qT1_sim_path_d2_d1_d10_multi_y[k]]).T.reshape(-1, 1, 3)
    segments = np.concatenate([points[:-2],points[1:-1], points[2:]], axis=1)
    
    lc = Line3DCollection(segments, cmap='viridis')
    # Set the values used for colormapping
    lc.set_array(qT1_sim_path_d2_d1_d10_multi_y[k])
    lc.set_linewidth(4)
    lc.set_clim(vmin=min(qT1_sim_path_d2_d1_d10_multi_y[k]), vmax=max(qT1_sim_path_d2_d1_d10_multi_y[k]))
                
    ax.add_collection3d(lc)
    #ax.set_title('middle')
    #ax.set(zlabel='qT1 [ms]')
    #ax.set(xlabel='geodesic distance\n[mm]')
    #ax.set(ylabel='path number')
    ax.set_xlabel('geodesic distance [mm]\ninferior > superior',labelpad=30, fontsize=20)
    ax.set_ylabel('path number\nanterior > posterior',labelpad=30, fontsize=20)
    ax.set_zlabel('qT1\n[ms]',labelpad=30, fontsize=20)
    ax.tick_params(axis='both', labelsize=20,pad=5,width=25)
    ax.tick_params(axis='z', labelsize=20,pad=15)
    ax.set_ylim(min(yrange),max(yrange))
    ax.set_zlim(2800,1200)
    ax.set_zticks([2800,1200])
    ax.set_yticks([0,4])
    ax.set_xticks([round(min(distance_f_h_all_d2_d1_d10_multi_y[k])-0.5,1),0,round(max(distance_f_h_all_d2_d1_d10_multi_y[k])+0.5,1)])
    ax.set_xlim(round(min(distance_f_h_all_d2_d1_d10_multi_y[k])-0.5,1),round(max(distance_f_h_all_d2_d1_d10_multi_y[k])+0.5,1))
    
    ax.plot(0, k, qT1_sim_path_d2_d1_d10_multi_y[k][np.where(distance_f_h_all_d2_d1_d10_multi_y[k]==0)[0][0]], color='r', marker='.', markersize=14)
#fig.colorbar(lc)
fig.tight_layout()

# save plots
############
fig.savefig(outdir+ind+'_3D_multi_path_mulicolor_y_middle_smooth2.png')
fig.savefig(outdir+ind+'_3D_multi_path_mulicolor_y_middle_smooth2.svg')

In [ ]:
# publication-ready 3D plots of multiple paths for middle along x-axis
######################################################################

from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import PolyCollection
from matplotlib import rcParams

rcParams['axes.labelpad'] = 25

# set up a figure twice as wide as it is tall
fig = plt.figure(figsize=plt.figaspect(8))

# =============
# First subplot
# =============
# set up the axes for the first plot
ax = fig.add_subplot(3, 1, 1, projection='3d', computed_zorder=False)

i = 0

yrange = [0,1,2,3,4]
verts = []

for k in reversed(yrange):
    
    yvalues = [k]*len(distance_f_h_all_d2_d1_d10_multi_x[k])
    points = np.array([distance_f_h_all_d2_d1_d10_multi_x[k], yvalues, qT1_sim_path_d2_d1_d10_multi_x[k]]).T.reshape(-1, 1, 3)
    segments = np.concatenate([points[:-2],points[1:-1], points[2:]], axis=1)
    
    lc = Line3DCollection(segments, cmap='viridis')
    # Set the values used for colormapping
    lc.set_array(qT1_sim_path_d2_d1_d10_multi_x[k])
    lc.set_linewidth(4)
    lc.set_clim(vmin=min(qT1_sim_path_d2_d1_d10_multi_x[k]), vmax=max(qT1_sim_path_d2_d1_d10_multi_x[k]))
                
    ax.add_collection3d(lc)
    #ax.set_title('middle')
    #ax.set(zlabel='qT1 [ms]')
    #ax.set(xlabel='geodesic distance\n[mm]')
    #ax.set(ylabel='path number')
    ax.set_xlabel('geodesic distance [mm]\ninferior > superior',labelpad=30, fontsize=20)
    ax.set_ylabel('path number\nanterior > posterior',labelpad=30, fontsize=20)
    ax.set_zlabel('qT1\n[ms]',labelpad=30, fontsize=20)
    ax.tick_params(axis='both', labelsize=20,pad=5,width=25)
    ax.tick_params(axis='z', labelsize=20,pad=15)
    ax.set_ylim(min(yrange),max(yrange))
    ax.set_zlim(2800,1200)
    ax.set_zticks([2800,1200])
    ax.set_yticks([0,4])
    ax.set_xticks([round(min(distance_f_h_all_d2_d1_d10_multi_x[k])-0.5,1),0,round(max(distance_f_h_all_d2_d1_d10_multi_x[k])+0.5,1)])
    ax.set_xlim(round(min(distance_f_h_all_d2_d1_d10_multi_x[k])-0.5,1),round(max(distance_f_h_all_d2_d1_d10_multi_x[k])+0.5,1))
    
    ax.plot(0, k, qT1_sim_path_d2_d1_d10_multi_x[k][np.where(distance_f_h_all_d2_d1_d10_multi_x[k]==0)[0][0]], color='r', marker='.', markersize=14)
#fig.colorbar(lc)
fig.tight_layout()

# save plots
############
fig.savefig(outdir+ind+'_3D_multi_path_mulicolor_x_middle_smooth.png')
fig.savefig(outdir+ind+'_3D_multi_path_mulicolor_x_middle_smooth.svg')

In [ ]:
#find most prominent peaks on geodesic paths sampled between forehead and little finger
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import PolyCollection
from matplotlib import rcParams

from scipy.signal import find_peaks, peak_prominences

peaks_m = [[0],[0],[0],[0],[0]]
properties_m = [[0],[0],[0],[0],[0]]

rcParams['axes.labelpad'] = 25

# set up a figure twice as wide as it is tall
fig, axs = plt.subplots(5, 1, sharex=True)

i = 0
yrange = [0,1,2,3,4]

# plot data
for k in reversed(yrange):
    print(k)
    print(np.std(qT1_sim_path_d2_d1_d10_multi_y[k]))
    peaks_m[k], properties_m[k] = find_peaks(qT1_sim_path_d2_d1_d10_multi_y[k],prominence=2*np.std(qT1_sim_path_d2_d1_d10_multi_y[k]),width=1,height=0,plateau_size=0,threshold=0)

    axs[k ].plot(distance_f_h_all_d2_d1_d10_multi_y[k],qT1_sim_path_d2_d1_d10_multi_y[k])
    axs[k ].plot(distance_f_h_all_d2_d1_d10_multi_y[k][peaks_m[k]], qT1_sim_path_d2_d1_d10_multi_y[k][peaks_m[k]], "x")
    axs[k ].vlines(x = distance_f_h_all_d2_d1_d10_multi_y[k][peaks_m[k]], ymin = qT1_sim_path_d2_d1_d10_multi_y[k][peaks_m[k]] - properties_m[k]["prominences"], ymax = qT1_sim_path_d2_d1_d10_multi_y[k][peaks_m[k]], color = "C1")

    axs[k ].plot(distance_f_h_all_d2_d1_d10_multi_y[k][0], qT1_sim_path_d2_d1_d10_multi_y[k][0], color='c', marker='.', markersize=10)
    axs[k ].plot(distance_f_h_all_d2_d1_d10_multi_y[k][-1], qT1_sim_path_d2_d1_d10_multi_y[k][-1], color='y', marker='.', markersize=10)
    axs[k ].plot(0, qT1_sim_path_d2_d1_d10_multi_y[k][np.where(distance_f_h_all_d2_d1_d10_multi_y[k]==0)[0][0]], color='r', marker='.', markersize=10)
    #axs[k ].plot(distance_f_h_all_d2_d1_d10_multi_y[k][len(qT1_sim_path_f_h_full_multi_y[k])+len(qT1_sim_path_h_d2_full_multi_y[k])-1], qT1_sim_path_d2_d1_d10_multi_y[k][len(qT1_sim_path_f_h_full_multi_y[k])+len(qT1_sim_path_h_d2_full_multi_y[k])-1], color='m', marker='.', markersize=10)
    axs[k ].invert_yaxis()
    
axs[k ].set_title('Middle Layer')

fig.savefig(outdir+ind+'_myelin_peaks_middle_smoothed.png')  

In [ ]:
#plot detrended qT1 values against normalized geodesic distances in one subplot 
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import PolyCollection
from matplotlib import rcParams

from scipy.signal import find_peaks, peak_prominences, detrend, normalize

rcParams['axes.labelpad'] = 25

fig, axs = plt.subplots(sharex=True)

i = 0

yrange = [0,1,2,3,4]

detrended_signal = []

for k in reversed(yrange):
    normalizedData = (distance_f_h_all_d2_d1_d10_multi_y[k]-np.min(distance_f_h_all_d2_d1_d10_multi_y[k]))/(np.max(distance_f_h_all_d2_d1_d10_multi_y[k])-np.min(distance_f_h_all_d2_d1_d10_multi_y[k]))
    axs.plot(normalizedData,detrend(qT1_sim_path_d2_d1_d10_multi_y[k]))
    axs.invert_yaxis()

axs.set_title('Middle Layer')

fig.savefig(outdir+ind+'_detrended_myelin_peaks_middle_norm_geod_smoothed.png') 

In [ ]:
print(qT1_sim_path_d2_d1_d10_multi_y)

In [ ]:
print(distance_f_h_all_d2_d1_d10_multi_y)

In [ ]:
def detrend_signal(signal):
    detrended_signal = []

    for k in range(len(signal)):
        detrended_signal.append(detrend(signal[k]))

    print("Detrended Signal: ", detrended_signal)
    return detrended_signal

detrended_signal = detrend_signal(qT1_sim_path_d2_d1_d10_multi_y)

In [ ]:
# plot detrended qT1 signal and detected peaks against normalized geodesic distances
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import PolyCollection
from matplotlib import rcParams

from scipy.signal import find_peaks, peak_prominences, detrend, normalize

yvalues = []
peaks_m = [[0],[0],[0],[0],[0]]
properties_m = [[0],[0],[0],[0],[0]]

#peaks_m[k][maxidx_prominences[k]]

rcParams['axes.labelpad'] = 25

# set up a figure twice as wide as it is tall
fig, axs = plt.subplots(5, 1, sharex=True)

i = 0

yrange = [0,1,2,3,4]
verts = []

# plot data
for k in reversed(yrange):
    
    # normalize geodesic distances to detect nearest peaks on neighboring paths  
    normalizedData = (distance_f_h_all_d2_d1_d10_multi_y[k]-np.min(distance_f_h_all_d2_d1_d10_multi_y[k]))/(np.max(distance_f_h_all_d2_d1_d10_multi_y[k])-np.min(distance_f_h_all_d2_d1_d10_multi_y[k]))
    
    peaks_m[k], properties_m[k] = find_peaks(detrended_signal[k],prominence=2*np.std(abs(detrended_signal[k])),width=1,height=0,plateau_size=0,threshold=0)
    zeropoint = np.where(distance_f_h_all_d2_d1_d10_multi_y[k]==0)[0][0]
  
    axs[k ].plot(normalizedData,detrended_signal[k])
    axs[k ].plot(normalizedData[peaks_m[k]], detrended_signal[k][peaks_m[k]], "x")
    axs[k ].vlines(x =  normalizedData[peaks_m[k]], ymin = detrended_signal[k][peaks_m[k]] - properties_m[k]["prominences"], ymax = detrended_signal[k][peaks_m[k]], color = "C1")

    axs[k ].plot(normalizedData[0], detrended_signal[k][0], color='c', marker='.', markersize=10)
    axs[k ].plot(normalizedData[-1], detrended_signal[k][-1], color='y', marker='.', markersize=10)
    axs[k ].plot(normalizedData[zeropoint], detrended_signal[k][np.where(distance_f_h_all_d2_d1_d10_multi_y[k]==0)[0][0]], color='r', marker='.', markersize=10)
    #axs[k ].plot(distance_f_h_all_d2_d1_d10_multi_y[k][len(qT1_sim_path_f_h_full_multi_y[k])+len(qT1_sim_path_h_d2_full_multi_y[k])-1], detrended_signal[k][len(qT1_sim_path_f_h_full_multi_y[k])+len(qT1_sim_path_h_d2_full_multi_y[k])-1], color='m', marker='.', markersize=10)
    axs[k ].invert_yaxis()
    
axs[k ].set_title('Middle Layer')

fig.savefig(outdir+ind+'_detrended_myelin_peaks_middle_smoothed_norm.png')  

# set up a figure twice as wide as it is tall
fig, axs = plt.subplots(5, 1, sharex=True)

i = 0

yrange = [0,1,2,3,4]
verts = []

# plot data
for k in reversed(yrange):
    
    # normalize geodesic distances to detect nearest peaks on neighboring paths  
    normalizedData = distance_f_h_all_d2_d1_d10_multi_y[k]
    
    peaks_m[k], properties_m[k] = find_peaks(detrended_signal[k],prominence=2*np.std(abs(detrended_signal[k])),width=1,height=0,plateau_size=0,threshold=0)
    zeropoint = np.where(distance_f_h_all_d2_d1_d10_multi_y[k]==0)[0][0]
  
    axs[k ].plot(normalizedData,detrended_signal[k])
    axs[k ].plot(normalizedData[peaks_m[k]], detrended_signal[k][peaks_m[k]], "x")
    axs[k ].vlines(x =  normalizedData[peaks_m[k]], ymin = detrended_signal[k][peaks_m[k]] - properties_m[k]["prominences"], ymax = detrended_signal[k][peaks_m[k]], color = "C1")

    axs[k ].plot(normalizedData[0], detrended_signal[k][0], color='c', marker='.', markersize=10)
    axs[k ].plot(normalizedData[-1], detrended_signal[k][-1], color='y', marker='.', markersize=10)
    axs[k ].plot(normalizedData[zeropoint], detrended_signal[k][np.where(distance_f_h_all_d2_d1_d10_multi_y[k]==0)[0][0]], color='r', marker='.', markersize=10)
    #axs[k ].plot(distance_f_h_all_d2_d1_d10_multi_y[k][len(qT1_sim_path_f_h_full_multi_y[k])+len(qT1_sim_path_h_d2_full_multi_y[k])-1], detrended_signal[k][len(qT1_sim_path_f_h_full_multi_y[k])+len(qT1_sim_path_h_d2_full_multi_y[k])-1], color='m', marker='.', markersize=10)
    axs[k ].invert_yaxis()
    
axs[k ].set_title('Middle Layer')

fig.savefig(outdir+ind+'_detrended_myelin_peaks_middle_smoothed.png')  

In [ ]:
print(distance_f_h_all_d2_d1_d10_multi_y)

In [ ]:
tmp = distance_f_h_all_d2_d1_d10_multi_y

def normalize_data(signals):
    
    normalizedData = [[] for i in list(range(0,len(signals)))]
    signal_tmp = [[] for i in list(range(0,len(signals)))]
    
    for k in range(0,len(signals)):
        # normalize geodesic distances to detect nearest peaks on neighboring paths  
        #normalizedData[k] = (signals[k]-np.min(signals[k]))/(np.max(signals[k])-np.min(signals[k]))
        normalizedData[k] = signals[k]
        normalizedData[k] = normalizedData[k].tolist()
    
    tmpi = signals
    
    print(normalizedData)
        
    minval = min(min(normalizedData))
    print(minval)
    
    for signal in range(0,len(tmpi)):
        res = [x + abs(minval) for x in tmpi[signal]]
        signal_tmp[signal] = res
    
    Data_type = object
    
    #make distances positive
    normalizedDataf = np.array(signal_tmp, dtype=Data_type)
    
    print(normalizedDataf)
    return normalizedDataf

normalizedData = normalize_data(tmp)

In [ ]:
print(distance_f_h_all_d2_d1_d10_multi_y)

In [ ]:
from scipy.interpolate import interp1d
from scipy.stats import sem
from peakutils import indexes
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from itertools import combinations

from scipy.stats import chi2
from matplotlib import patches
import matplotlib.pyplot as plt

def define_comparisons(signals,peaks_m):
    
    signal_numbers = list(range(0,len(signals)))
    print(signal_numbers)
    
    #signal_pairs = list(combinations(signal_numbers, 2))
    #signal_pairs = list([[signals[0],signals[1]],[signals[1],signals[2]],[signals[2],signals[3]],[signals[3],signals[4]],[signals[0],signals[2]],[signals[1],signals[3]],[signals[2],signals[4]]])
    signal_pairs = list([[0,1],[1,2],[2,3],[3,4],[0,2],[1,3],[2,4]])
    print(signal_pairs)
    
    print(len(signals))
    
    #signal_pairs = [i for i in signal_pairs if i != (0,len(signals)-1)]
    #print(signal_pairs)
    
    peak_idx_pairs = [[] for i in list(range(0,len(signal_pairs)))]
    print(peak_idx_pairs)
    
    print("Peaks_m: ")
    print(peaks_m[0])
    
    #iterate over each signal and sort peaks according to signal pairs
    for i in range(0,len(signals)):      
        print(list(peaks_m[0][i]))
        peak = list(peaks_m[0][i])
        
        for k in range(0,len(signal_pairs)):
            if signal_pairs[k][0] == i or signal_pairs[k][1] == i:
                peak_idx_pairs[k].append(peak)
    
    print("Peak_idx_pairs: ")
    print(peak_idx_pairs)
    
    return peak_idx_pairs, signal_pairs

def find_prominent_peaks(signals): #find peaks in original signal, project peaks to DTW space and calculate intersections

    peaks_m = [[0]]*len(signals)
    properties_m = [[0]]*len(signals)
    
    for i in range(len(signals)):
        peaks_m[i], properties_m[i] = find_peaks(signals[i],prominence=2*np.std(abs(signals[i])),width=1,height=0,plateau_size=0,threshold=0)

    return peaks_m, properties_m


def get_border_params(combined_final_borders,peaks_m,signals,orig_signals):    
    
    border_loc = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
    path_id = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
    borderp = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
    borderh = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
    borderw = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
    borderintens = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
    borderorigintens = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
    borderqsm = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
    borderaqsm = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
    borderpqsm = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
    bordernqsm = np.empty((5,5,)) * np.nan # 5 paths x 5 borders    

    borderf = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
    borderh = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
    
    #extract border parameters
    count = 0
    vcount = 0
    for border in combined_final_borders: #loop over all detected borders
        vcount = 0
        for valley in border: #for each border loop over all detected valleys that shape this specific border
            border_loc[count][vcount] = valley[1]
            path_id[count][vcount] = valley[0]
            borderp[count][vcount] = peaks_m[1][valley[0]]["prominences"][np.where(peaks_m[0][valley[0]]==valley[1])[0]]
            borderh[count][vcount] = peaks_m[1][valley[0]]["peak_heights"][np.where(peaks_m[0][valley[0]]==valley[1])[0]]
            borderw[count][vcount] = peaks_m[1][valley[0]]["widths"][np.where(peaks_m[0][valley[0]]==valley[1])[0]]
            borderintens[count][vcount] = signals[valley[0]][valley[1]]
            borderorigintens[count][vcount] = orig_signals[valley[0]][valley[1]]
            vcount = vcount + 1
        count = count + 1
    
    border_loc = border_loc.transpose()
    path_id = path_id.transpose()
    borderp = borderp.transpose()
    borderh = borderh.transpose()
    borderw = borderw.transpose()
    borderintens = borderintens.transpose()
    borderorigintens = borderorigintens.transpose()
    borderqsm = borderqsm.transpose()
    borderaqsm = borderaqsm.transpose()
    borderpqsm = borderpqsm.transpose()
    bordernqsm = bordernqsm.transpose()

    borderf = borderf.transpose()
    borderh = borderh.transpose()
    
    np.savetxt(outdir+ind+"_border_locations_middle_multi_y_paths.csv", border_loc, delimiter=" ")
    np.savetxt(outdir+ind+"_path_id_middle_multi_y_paths.csv", path_id, delimiter=" ")
    np.savetxt(outdir+ind+"_border_prominences_middle_multi_y_paths.csv", borderp, delimiter=" ")
    np.savetxt(outdir+ind+"_border_widths_middle_multi_y_paths.csv", borderw, delimiter=" ")
    np.savetxt(outdir+ind+"_border_heights_middle_multi_y_paths.csv", borderh, delimiter=" ")
    np.savetxt(outdir+ind+"_border_intensity_middle_multi_y_paths.csv", borderintens, delimiter=" ")
    np.savetxt(outdir+ind+"_border_origintensity_middle_multi_y_paths.csv", borderorigintens, delimiter=" ")

    combined_border_params = np.concatenate((path_id, border_loc, borderw, borderh, borderp, borderintens, borderorigintens, borderqsm, borderaqsm, borderpqsm, bordernqsm, borderf, borderh), axis=1)
    print(combined_border_params)

    np.savetxt(outdir+ind+"_border_params_singlepaths_middle_multi_y_paths.csv", combined_border_params, delimiter=" ")

    mean_combined_border_params = np.nanmean(combined_border_params,axis=0)
    print(mean_combined_border_params)
    
    border_count = len(combined_final_borders)
    
    final_mean_combined_border_params = np.insert(mean_combined_border_params,0,border_count)
    final_mean_combined_border_params = np.insert(final_mean_combined_border_params,0,ind_i)
    print(final_mean_combined_border_params)

    # create column headers for table and save header file
    header = np.array([['ID','border_count','path1','path2','path3','path4','path5','borderloc1','borderloc2','borderloc3','borderloc4','borderloc5','width1','width2','width3','width4','width5','height1','height2','height3','height4','height5','prom1','prom2','prom3','prom4','prom5','intens1','intens2','intens3','intens4','intens5','origintens1','origintens2','origintens3','origintens4','origintens5','borderqsm1','borderqsm2','borderqsm3','borderqsm4','borderqsm5','borderaqsm1','borderaqsm2','borderaqsm3','borderaqsm4','borderaqsm5','borderpqsm1','borderpqsm2','borderpqsm3','borderpqsm4','borderpqsm5','bordernqsm1','bordernqsm2','bordernqsm3','bordernqsm4','bordernqsm5','borderspmTf1','borderspmTh1','borderspmTf2','borderspmTh2','borderspmTf3','borderspmTh3','borderspmTf4','borderspmTh4','borderspmTf5','borderspmTh5']])
    print(header[0])
    np.savetxt(outdir+ind+"_border_header_middle_multi_y_paths.csv", header, delimiter=" ", fmt="%s")

    header_str = "ID,border_count,path1,path2,path3,path4,path5,borderloc1,borderloc2,borderloc3,borderloc4,borderloc5,width1,width2,width3,width4,width5,height1,height2,height3,height4,height5,prom1,prom2,prom3,prom4,prom5,intens1,intens2,intens3,intens4,intens5,origintens1,origintens2,origintens3,origintens4,origintens5,borderqsm1,borderqsm2,borderqsm3,borderqsm4,borderqsm5,borderaqsm1,borderaqsm2,borderaqsm3,borderaqsm4,borderaqsm5,borderpqsm1,borderpqsm2,borderpqsm3,borderpqsm4,borderpqsm5,bordernqsm1,bordernqsm2,bordernqsm3,bordernqsm4,bordernqsm5,borderspmTf1,borderspmTh1,borderspmTf2,borderspmTh2,borderspmTf3,borderspmTh3,borderspmTf4,borderspmTh4,borderspmTf5,borderspmTh5"

    # save border parameter values (from anterior to posterior)
    # includes: number of detected borders, single widths, single heights, single prominences, single plateau sizes, single left thresholds, single right thresholds, single border width heights
    border_params = np.array([final_mean_combined_border_params])
    print(border_params)
    np.savetxt(outdir+ind+"_border_params_mean_middle_multi_y_paths.csv", border_params, delimiter=" ")
    np.savetxt(outdir+ind+"_border_params_header_middle_multi_y_paths.csv", border_params, header=header_str, delimiter=" ")

    border_params_allsubj = np.genfromtxt(outdir_all+"allsubj_border_params_middle_multi_y_paths.csv",delimiter=",")
    print(border_params_allsubj)
    border_params_allsubj[ind_i] = border_params
    print(border_params_allsubj[ind_i])
    print(ind_i)

    np.savetxt(outdir_all+"allsubj_border_params_middle_multi_y_paths.csv", border_params_allsubj, delimiter=",")   
    
    return border_params,header

def plot_signals(peaks_m_unq,signals,xdata,fhpath,d2path,datatype,peaks_m):

    colors = ["darkorange","darkcyan","darkviolet","darkred","darkgreen","darkblue"]
    
    if len(signals) > 1:
        fig, axs = plt.subplots(len(signals), 1, sharex=True)
    else:
        fig, axs = plt.subplots(len(signals)+1, 1, sharex=True)
        
    for k in range(len(signals)):
        # plot qT1 signal against geodesic distances
        axs[k ].plot(xdata[k],signals[k])
        axs[k ].invert_yaxis()
        
        peak_color = 0
        
        for peaks in peaks_m_unq: 
            
            for peak in peaks:
                if peak[0] == k:
                    axs[k ].plot(xdata[peak[0]][peak[1]],signals[peak[0]][peak[1]],marker="x",markeredgewidth=2,color=colors[peak_color])
            peak_color = peak_color + 1
    
    fig.savefig(outdir+ind+'_'+datatype+'_qT1_signals_final_peaks.png')
    
    
    if len(signals) > 1:
        fig, axs = plt.subplots(len(signals), 1, sharex=True)
    else:
        fig, axs = plt.subplots(len(signals)+1, 1, sharex=True)

    for k in range(len(signals)):
        # plot qT1 signal against geodesic distances
        axs[k ].plot(xdata[k],signals[k])
        axs[k ].invert_yaxis()
        
        peak_color = 0
        
        axs[k ].plot(xdata[k][0], signals[k][0], color='c', marker='.', markersize=10)
        axs[k ].plot(xdata[k][-1], signals[k][-1], color='y', marker='.', markersize=10)
        axs[k ].plot(0, signals[k][np.where(xdata[k]==0)[0][0]], color='r', marker='.', markersize=10)
        #axs[k ].plot(0, signals[k][np.where(xdata[k]==0)[0][0]], color='r', marker='.', markersize=10)
        
        for peaks in peaks_m_unq: 
            for peak in peaks:
                if peak[0] == k:
                    axs[k ].plot(xdata[peak[0]][peak[1]],signals[peak[0]][peak[1]],marker="x",markeredgewidth=2,color=colors[peak_color])
                    axs[k ].vlines(x = xdata[peak[0]][peak[1]], ymin = signals[peak[0]][peak[1]] - peaks_m[1][peak[0]]["prominences"][np.where(peaks_m[0][peak[0]]==peak[1])[0]], ymax = signals[peak[0]][peak[1]], color=colors[peak_color])

            peak_color = peak_color + 1
            
        #if datatype != "detrended":
        #    axs[k ].plot(distance_f_h_all_d2_d1_d10_multi_y[k][len(fhpath[k])+len(d2path[k])-1], qT1_sim_path_d2_d1_d10_multi_y[k][len(fhpath[k])+len(d2path[k])-1], color='m', marker='.', markersize=10)

    fig.savefig(outdir+ind+'_'+datatype+'_qT1_signals_final_peaks_seeds.png')

In [ ]:
# save signal peaks and properties to file
# add border_count param to file

detrended_signal = detrend_signal(qT1_sim_path_d2_d1_d10_multi_y)

In [ ]:
peaks_m = find_prominent_peaks(detrended_signal)
print(peaks_m)

In [ ]:
peak_idx_pairs, signal_pairs = define_comparisons(detrended_signal,peaks_m)

In [ ]:
#detect borders in the smoothed qT1 signal along normalized geodesic distances
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import PolyCollection
from matplotlib import rcParams

import more_itertools

from itertools import chain

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


def init_peak_pairs(peak_idx_pairs, signal_pairs, normalizedData):
    yvalues = []
    peaks_m_final = []
    peaks_m_next_final = []

    final = [0,0,0,0,0]

    i = 0

    pairs = []

    count = 0

    for k in range(0,len(signal_pairs)):
        print(len(peak_idx_pairs))
        print(len(signal_pairs))
        #print number of detected peaks:
        print("# of Detected Peaks along path " + str(signal_pairs[k][0]) + ": " + str(len(peak_idx_pairs[k][0])))
        print("# of Detected Peaks along path " + str(signal_pairs[k][1]) + ": " + str(len(peak_idx_pairs[k][1])))

        index = []
        mindiff = []
        x = []
        xval = []

        # find nearest peaks on neighboring geodesic path by comparing differences in normalized geodesic distance:
        arr = np.array(normalizedData[signal_pairs[k][1]],dtype=object)
        arr = arr[peak_idx_pairs[k][1]]
        print("Array is : ", arr)

        for l in range(0,len(peak_idx_pairs[k][0])):
            count = count + 1
            print(count)
            # element to which nearest value is to be found
            x.append(l)
            xval.append(normalizedData[signal_pairs[k][0]][peak_idx_pairs[k][0][l]])
            print("Value to which nearest element is to be found: ", xval[l])

            # calculate the difference array
            difference_array = np.absolute(arr-xval[l])
            signed_difference_array = arr-xval[l]

            print("signed_difference_array")
            print(signed_difference_array)
            
            tmp_arr = signed_difference_array[signed_difference_array>0]
            
            if len(tmp_arr) == 0:
                print("ONLY NEGATIVE VALUES!!")
            print(tmp_arr)

            # find the index of minimum element from the array
            try:
                if subj==0:
                    minidx = difference_array.argmin()
                    if signal_pairs[k][0] < signal_pairs[k][1] and abs(signal_pairs[k][0]-signal_pairs[k][1]) > 1 and signed_difference_array[minidx] < -2.5: 
                        index.append(list(np.where(signed_difference_array==[min(signed_difference_array[signed_difference_array>0])]))[0][0])
                    elif signal_pairs[k][0] > signal_pairs[k][1] and abs(signal_pairs[k][0]-signal_pairs[k][1]) > 1 and signed_difference_array[minidx] > 2.5: 
                        index.append(list(np.where(signed_difference_array==[max(signed_difference_array[signed_difference_array<0])]))[0][0])
                    elif signal_pairs[k][0] < signal_pairs[k][1] and abs(signal_pairs[k][0]-signal_pairs[k][1]) == 1 and signed_difference_array[minidx] < -2.5 and difference_array[minidx] > 2.5: 
                        index.append(list(np.where(signed_difference_array==[min(signed_difference_array[signed_difference_array>0])]))[0][0])
                    else:
                        index.append(minidx)
                else:
                    #difference_array = abs(signed_difference_array)
                    minidx = difference_array.argmin()
                    if signal_pairs[k][0] < signal_pairs[k][1] and abs(signal_pairs[k][0]-signal_pairs[k][1]) > 1 and signed_difference_array[minidx] < -2.5: 
                        index.append(list(np.where(signed_difference_array==[min(signed_difference_array[signed_difference_array>0])]))[0][0])
                    elif signal_pairs[k][0] > signal_pairs[k][1] and abs(signal_pairs[k][0]-signal_pairs[k][1]) > 1 and signed_difference_array[minidx] > 2.5: 
                        index.append(list(np.where(signed_difference_array==[max(signed_difference_array[signed_difference_array<0])]))[0][0])
                    elif signal_pairs[k][0] < signal_pairs[k][1] and abs(signal_pairs[k][0]-signal_pairs[k][1]) == 1 and signed_difference_array[minidx] < -2.5 and difference_array[minidx] > 2.5: 
                        index.append(list(np.where(signed_difference_array==[min(signed_difference_array[signed_difference_array>0])]))[0][0])
                    else:
                        index.append(minidx)
                    
                print("Nearest element to the given values is : ", arr[index[l]])
                print("Index of nearest value is : ", index[l])

                print("Difference between position of actual value and next value is: ", difference_array[index[l]])
                mindiff.append(difference_array[index[l]])

                pairs.append([(signal_pairs[k][0],l),(signal_pairs[k][1],index[l]),(difference_array[index[l]]),(signed_difference_array[index[l]])])
            except:
                print('No valid near neighbor can be detected.')

        # only store index of peak if its distance is closer than 0.3 (1/3 of the whole path length) to the previous detected peak
        print("Initial Pairs:")
        print(pairs)
        mindiff_03_idx = np.where(np.array(mindiff)<5.0)[0].tolist()
        print(mindiff)
        print(mindiff_03_idx)

        print("Peaks Next: ", index)
        peaks_m_next_final.append(np.take(index, mindiff_03_idx).tolist()) 
        print("Peaks Next Final: ",peaks_m_next_final)

        unq, unq_idx, unq_cnt = np.unique(index, return_inverse=True, return_counts=True)
        cnt_mask = unq_cnt > 1
        cnt_unique = unq_cnt == 1
        uniques = unq[cnt_unique]
        duplicates = unq[cnt_mask]

        x_idx = []

        for i in range(len(uniques)):
            uniques_idx = np.where(index==uniques[i])[0]
            print("uniques_idx: ", uniques_idx)
            x_idx.append(uniques_idx)

        for i in range(len(duplicates)):
            duplicates_idx = np.where(index==duplicates[i])[0]
            mindiff_min_idx = np.take(mindiff,duplicates_idx).argmin()
            x_idx.append(np.take(duplicates_idx,mindiff_min_idx))

        flat_x_idx = list(more_itertools.collapse(x_idx))
        print("Flat list: ",flat_x_idx)

        fin_x_idx = []
        for i in flat_x_idx:
            print(i)
            if mindiff[i] < 5.0:
                fin_x_idx.append(x[i])

        peaks_m_final.append(fin_x_idx)

    print(count)
    print("Pairs: ", pairs)
    for r in pairs:
        if r[2] > 5.0:
            pairs.remove(r)
    print("Pairs after removing pairs of high diffs: ", pairs)
    
    return pairs

pairs = init_peak_pairs(peak_idx_pairs, signal_pairs, normalizedData)

copy_pairs = pairs

init_pairs = [sublist[0:2] for sublist in pairs]
print(init_pairs)

In [ ]:
def find_intersections(pairs):
    # check for all 1st valleys which have the same 2nd valley and only keep those valley pairs with the minimum distance
    last_valleys = [sublist[-3] for sublist in pairs]
    duplicates=[]
    indices=[]
    count = 0
    for i, tuple_ in enumerate(last_valleys):
        print(i)
        print("Tuple_: ",tuple_)
        pairs[i].append(i)
        if last_valleys.count(tuple_) > 1:
            duplicates.append(pairs[i])
            print(duplicates)
            print(duplicates[count])
            #duplicates[count].append(i)
            indices.append(i)
            count = count + 1
    print("Sublists with duplicate 2nd valleys: ", duplicates)
    print("Indices of Duplicates: ", indices)

    maxvalley2_idx = []
    grouped_sublists = {}

    for sublist in duplicates:
        last_tuple = sublist[1]
        if last_tuple in grouped_sublists:
            grouped_sublists[last_tuple].append(sublist)
        else:
            grouped_sublists[last_tuple] = [sublist]
    print(grouped_sublists)
    print(len(grouped_sublists.items()))

    maxvalley2 = []
    print(maxvalley2)
    #count = 0

    for key,value in grouped_sublists.items():
        print(value)
        #print(count)
        #print(maxvalley2[count])
        diffvalues = []
        for i in range(len(value)):
            print(len(value))
            print(value[i][0])
            print(value[i][-3])
            diffvalues.append(value[i][0:4])
        maxvalley2.append(diffvalues)
        #count = count + 1
    print("Maxvalley2:")
    print(maxvalley2)

    for i in range(len(maxvalley2)):
        
        if subj==0:
            for dupl in maxvalley2[i]:
            ### more advanced decision criterion on which valleys are to be removed ###
                if dupl[0][0] < dupl[1][0] and abs(dupl[0][0]-dupl[1][0]) > 1 and dupl[3] < 0: 
                    dupl[2] = np.nan
                elif dupl[0][0] > dupl[1][0] and abs(dupl[0][0]-dupl[1][0]) > 1 and dupl[3] > 0: 
                    dupl[2] = np.nan

        abs_dist = [elem[2] for elem in maxvalley2[i]]

        print("Absolute Differences after setting wrong direction elements to nan")
        print(abs_dist)
        print(np.nanargmin(abs_dist))
        maxvalley2_idx.append(np.nanargmin(abs_dist))

    print("maxvalley2_idx")
    print(maxvalley2_idx)

    remove_valleys = []
    count = 0
    for key,value in grouped_sublists.items():
        print("Value:")
        print(value)
        rvalues = []
        value.pop(maxvalley2_idx[count])
        for i in range(len(value)):
            rvalues.append([value[i][-1]])
            #value[i].pop(-1)
        remove_valleys.append(rvalues)
        #print(value[count])
        print(value)
        count = count +1
    print("Valleys to be removed: ", remove_valleys)

    flat_remove_valley = list(more_itertools.collapse(remove_valleys))

    # remove sublists containing 2nd valley with higher distance from border pairs
    rpairs = [ele for idx, ele in enumerate(pairs) if idx not in flat_remove_valley]
    print("RPAIRS: ", rpairs)

    tmp = []
    rpairs_fin = []
    for elem in rpairs:
        if elem[2] > 4 and abs(elem[0][0]-elem[1][0]) > 1 and elem[0][0] < elem[1][0] and elem[3] < 0:
            tmp.append(elem)
        else:
            rpairs_fin.append(elem)

    print("rpairs_fin:")
    print(rpairs_fin)

    remove=[]
    for i in range(len(rpairs_fin)):
        if rpairs_fin[i][-3] > 5.0:
            remove.append(rpairs_fin[i])

    for i in remove:
        rpairs_fin.remove(i)
    print(("RPAIRS_fin: ", rpairs_fin))

    # remove diff and ID distance tuple from all pairs of remaining neighboring valleys
    fpairs = [sublist[:-3] for sublist in rpairs_fin]
    print("Pairs after removing diff tuple: ", fpairs)

    # create feature matrix of remaining valleys
    print("Valley Properties: ", properties_m)

    ufpairs = list(set(chain.from_iterable(fpairs)))
    print(ufpairs)

    prominence = []
    height = []
    width = []

    for valley in ufpairs:
        prominence.append(properties_m[valley[0]]["prominences"][valley[1]])
        height.append(properties_m[valley[0]]["peak_heights"][valley[1]])
        width.append(properties_m[valley[0]]["widths"][valley[1]])
    print("Prominences: ", prominence)
    
    return fpairs, ufpairs

fpairs, ufpairs = find_intersections(pairs)


In [ ]:
def merge_intersecting_lists(lists):
    import networkx as nx
    g = nx.Graph()

    for sub_list in lists:
        for i in range(1,len(sub_list)):
            g.add_edge(sub_list[0],sub_list[i])
    
    merged_list = list(nx.connected_components(g))
    print(merged_list)
    return merged_list

# merge valleys with intersections together to obtain groups of neighboring valleys which shape a border
merged_list = merge_intersecting_lists(fpairs)
print("Merged Lists: ", merged_list) #represent border groups
print(len(merged_list))

In [ ]:
# only those goups are considered as real borders which have a minmum length of 3 (i.e., occur across a minmum of 3 paths)
def check_min_length(merged_list):
    fmerged_list = merged_list
    len(fmerged_list)
    fmerged_remove = []
    for i in fmerged_list:
        if len(i) < 3:
            fmerged_remove.append(i)
    for remove in fmerged_remove:
        fmerged_list.remove(remove)
    return fmerged_list

fmerged_list = check_min_length(merged_list)
print("Final Merged List: ", fmerged_list)
print(len(fmerged_list))

In [ ]:
def get_original_peaks(result):
    result = list(result)
    for it in range(0,len(result)):
        result[it] = list(result[it])
        for i in range(0,len(result[it])):
            result[it][i] = list(result[it][i])
            result[it][i][1] = peaks_m[0][result[it][i][0]][result[it][i][1]]
    print(result)
    
    return result

result = get_original_peaks(fmerged_list)
    

In [ ]:
def check_border_peaks(result,number,normalizedData):
    number_signals = list(range(0,number))
    print(number_signals)
    count = [[] for i in range(0,number)]
    tmp = [[] for i in range(0,number)]
    i = 0
    multiple = False
    new_res = result
    
    print(result)
    
    rmv_check = [[] for i in range(0,number)]
    
    max_gdist = []
    
    for res in result:
        
        res.sort(key=lambda sublist: sublist[0])
        
        index_count = [[] for i in range(0,number)]
        rmv = []
        gdist = []        
        
        for num in number_signals:
            j = 0
            print("Signal Number:")
            print(num)
            count[i].append(sum(item[0]==num for item in res))
            print("Number of Peaks per Signal in Group:")
            print(count[i])
            
            if count[i][num] > 1:
                resfirst = [item[0] for item in res]
                for it in range(0,len(resfirst)):
                    if resfirst[it] == num:
                        index_count[num].append(it)
                    print("Index Count:")
                    print(index_count)
        
                #min(signed_difference_array[signed_difference_array>0])
        for index in index_count:
            print("INDEX")
            print(index)
            if len(index) > 0:
                print("testtest")
                print(res)
                tmp[i].append(np.array(res)[index].tolist())
            
            print("Duplicates:")
            print(tmp[i])
        
        for t in tmp[i]:
            gdist = []
            if len(t) > 1:
                multiple = True
                resfirst = [item[0] for item in res]
                ressecond = [item[1] for item in res]

                neighbors = abs(np.array(resfirst)-t[0][0])
                print("Neighbors to Duplicate:")
                print(neighbors)

                nearest_neighbor = np.where(neighbors == min(neighbors[neighbors>0]))[0][0]
                print('nearest_neighbor')
                print(nearest_neighbor)

                for tt in t:
                    print('tt')
                    print(tt)
                    gdist_tmp = normalizedData[tt[0]][tt[1]] - normalizedData[resfirst[nearest_neighbor]][ressecond[nearest_neighbor]]
                    
                    #if (tt[0] > resfirst[nearest_neighbor] and gdist_tmp > -2.49) or tt[0] < resfirst[nearest_neighbor]: 
                    if gdist_tmp < -2.5:
                        gdist.append(9999)
                    else:
                        gdist.append(gdist_tmp)
                
                print('gdist')       
                print(gdist)
                
                try:
                    max_gdist = np.argmax(np.absolute(gdist))
                    rmv.append(t[max_gdist])
                    print(max_gdist)
                except:
                    print('Nothing to remove. Try next.')
                    
                res = [ele for ele in res if ele not in rmv]
                
                    #print(resfirst[nearest_neighbor])
                    #print(t[0])
                    #if resfirst[nearest_neighbor] < t[0] and abs(resfirst[nearest_neighbor]-t[0]) > 0 and normalizedData[t[0]][t[1]] - normalizedData[resfirst[nearest_neighbor]][ressecond[nearest_neighbor]] < 0:
                    #    rmv.append(t)
                    #elif resfirst[nearest_neighbor] > t[0] and abs(resfirst[nearest_neighbor]-t[0]) > 0 and normalizedData[t[0]][t[1]] - normalizedData[resfirst[nearest_neighbor]][ressecond[nearest_neighbor]] > 0:
                    #    rmv.append(t)
        
        rmv_check[i] = rmv   
        
        new_res[i] = [ele for ele in res if ele not in rmv]
        print(new_res[i])
        
        new_res[i].sort()
        print(new_res[i])
                   
        if len(new_res[i]) < 3:
            new_res[i] = []            
            
        i = i +1
    
    result = new_res
    
    rmv = []
    
    y = 0
        
    for r in result:
        norm_result = []
        print(r)
        for i in r:
            print(i[0])
            print(normalizedData[i[0]][20])
            norm_result.append(normalizedData[i[0]][i[1]])
        
        #print(norm_result)
        
        grad_result = np.gradient(norm_result)
        
        print(grad_result)
        
        #print(abs(grad_result))
        
        #print(2*np.std(abs(grad_result)))
        
        #std_grad_result = 2*np.std(abs(grad_result))
        
        #print(np.gradient(grad_result))
        
        #print(np.std(np.gradient(grad_result)))
        
        # IQR
        # Find Q1, Q3
        # 1.
        #Q1 = np.percentile((grad_result) , 25)
        #Q3 = np.percentile((grad_result) , 75)

        # 2.
        #Q1,Q3 = np.percentile((abs(grad_result)) , [25,75])

        #Find IQR, upper limit, lower limit
        #IQR = Q3 - Q1
        #ul = Q3+1.5*IQR
        #ll = Q1-1.5*IQR
        
        #print(ul)
        #print(ll)

        abs_grad_result = abs(grad_result)
        
        # Find outliers
        #outliers = abs_grad_result[(abs_grad_result > ul) | (abs_grad_result < ll)]
        #outliers = list(np.where(abs_grad_result > std_grad_result))[0]
        outliers = np.where(abs_grad_result > 5)[0]
        print(outliers)
        
        for out in outliers:
            rmv_check[y].append(r[out])
        
        print(rmv)
        
        result[y] = [ele for ele in r if ele not in rmv_check[y]]
        
        y = y + 1
        
    for elem in range(0, len(result)):
        if len(result[elem]) < 3:
            result[elem] = []
        
    print(result)
    
    result = [ele for ele in result if len(ele) > 0]
    
    print(result)
    
    rmv_check.append(rmv)
    
    print(rmv_check)
        
    for elem in rmv_check:
        if len(elem) > 2:
            result.append(elem)
    
    print(result)
    
    sorted_result = sorted(result, key=lambda sublist: sublist[0][1])
    
    print(sorted_result)
    
    return sorted_result
                

In [ ]:
number = len(detrended_signal)
final_result = check_border_peaks(result,number,normalizedData)

# exctract number of detected borders
border_count = len(final_result)
print(border_count)

In [ ]:
datat = "raw"
plot_signals(final_result,qT1_sim_path_d2_d1_d10_multi_y,distance_f_h_all_d2_d1_d10_multi_y,qT1_sim_path_f_h_full_multi_y,qT1_sim_path_h_d2_full_multi_y,datat,peaks_m)

In [ ]:
distance_f_h_all_d2_d1_d10_multi_y

In [ ]:
datat = "detrended"
plot_signals(final_result,detrended_signal,distance_f_h_all_d2_d1_d10_multi_y,qT1_sim_path_f_h_full_multi_y,qT1_sim_path_h_d2_full_multi_y,datat,peaks_m)

In [ ]:
# map extracted borders back to surface
######################################################
# middle qT1

crgb = ["darkorange","darkcyan","darkviolet","darkred","darkgreen","darkblue"]
#crgb = ("r","c","g","y","b")

#border_loc_mesh = sum(result, [])
border_loc_mesh = final_result
print(border_loc_mesh)

merged_path = [pv.PolyData(),pv.PolyData(),pv.PolyData(),pv.PolyData(),pv.PolyData()]

for i in range(len(path_f_h_full_multi_yred)):
    
    final_path_length = len(path_f_h_full_multi_yred[i].points) + len(path_d2_d5_full_multi_yred[i].points) - 1
    print(len(path_f_h_full_multi_yred[i].points) + len(path_d2_d5_full_multi_yred[i].points) - 1)
    
    #merging in this way ensures that superior path is written to the end of the array and borders appear in the correct order on the surface
    merged_path[i] = path_d2_d5_full_multi_yred[i].merge(path_f_h_full_multi_yred[i])

    if final_path_length == len(merged_path[i].points):
        path_msg = "Border correctly projected. Correct final path langths."
    else:
        path_msg = "Borders may not be correctly projected. Mismatch in final path langths. Check merged_path variable."
    
    print(len(merged_path[i].points))
    
    print(path_d2_d5_full_multi_yred[i].points)
    print(merged_path[i].points)
    
print(merged_path)

yrange = [0,1,2,3,4]
i=0

#plot surface
p4 = pv.Plotter()
p4.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        print(border_loc_mesh[k])
        p4.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=6, color = crgb[k])
        print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
        p4.add_mesh(merged_path[border_loc_mesh[k][i][0]])
p4.camera_position = p4_camera_position
p4.show()

#save surface plot
p4 = pv.Plotter(off_screen=True)
p4.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p4.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=6, color = crgb[k])
        print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
p4.camera_position = p4_camera_position
p4.screenshot(outdir+ind+'_qT1_signal_final_borders_smooth.png',transparent_background=True)

In [ ]:
#p4_camera_position = [(93.96340522180034, 62.05208176751892, -4.599802458284934),
#(23.958131814789642, 78.3841590360701, 63.502972950631026),
#(-0.6601009969549828, -0.5029854388605619, -0.5579178453081473)]

#pv.global_theme.anti_aliasing = 'ssaa'

#plot surface
p4 = pv.Plotter()
p4.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p4.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=6, color = crgb[k])
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
p4.camera_position = p4_camera_position
p4.camera.zoom(2.0)
#p4.enable_anti_aliasing('msaa')
#p4.camera.view_frustum(1.0)
#p4.camera.view_angle = 20
p4.show()

#save surface plot
p4 = pv.Plotter(off_screen=True)
p4.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p4.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=6, color = crgb[k])
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
p4.camera_position = p4_camera_position
p4.camera.zoom(2.0)
p4.screenshot(outdir+ind+'_qT1_signal_final_borders_smooth_zoomed.png',transparent_background=True, window_size=(500,950))

In [ ]:
#p4_camera_position = [(93.96340522180034, 62.05208176751892, -4.599802458284934),
#(23.958131814789642, 78.3841590360701, 63.502972950631026),
#(-0.6601009969549828, -0.5029854388605619, -0.5579178453081473)]

#pv.global_theme.anti_aliasing = 'ssaa'

#plot surface
p4 = pv.Plotter()
p4.add_mesh(mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p4.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=6, color = crgb[k])
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
p4.camera_position = p4_camera_position
p4.camera.zoom(2.0)
#p4.enable_anti_aliasing('msaa')
#p4.camera.view_frustum(1.0)
#p4.camera.view_angle = 20
p4.show()

#save surface plot
p4 = pv.Plotter(off_screen=True)
p4.add_mesh(mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p4.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=6, color = crgb[k])
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
p4.camera_position = p4_camera_position
p4.camera.zoom(2.0)
p4.screenshot(outdir+ind+'_qT1_signal_final_borders_orig_zoomed.png',transparent_background=True, window_size=(500,950))

In [ ]:
border_params, header = get_border_params(final_result,peaks_m,qT1_sim_path_d2_d1_d10_multi_y,qT1_im_path_d2_d1_d10_multi_y)

In [ ]:
def write_points_to_vtk(array_of_polyData, points):
    
    combined_points = []
    mesh = []
    
    for k in range(len(points)):
        p4.add_points(array_of_polyData[points[k][0]].points[points[k][1],:], point_size=6, color = 'r')
        print(array_of_polyData[points[k][0]].points[points[k][1],:])
    
    for k in range(len(points)):
        combined_points.append(array_of_polyData[points[k][0]].points[points[k][1],:])
        mesh.append(array_of_polyData[points[k][0]].extract_points(points[k][1],adjacent_cells = False, include_cells = False))
    
    combined_mesh = mesh[0]
    for m in range(len(mesh)-1):
        combined_mesh = combined_mesh.merge(mesh[m+1])
    
    print(combined_points)
    print(mesh)
    print(combined_mesh)
    
    combined_mesh.save(outdir+ind+'_extracted_borders.vtk')

    return combined_mesh

border_loc_mesh_new = sum(list(border_loc_mesh), [])

try:
    mesh = write_points_to_vtk(merged_path,border_loc_mesh_new)
    #plot surface
    p4 = pv.Plotter()
    p4.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
    p4.add_mesh(mesh, point_size=6, color="m")
    p4.camera_position = p4_camera_position
    p4.show()
except:
    print("No Borders identified that can be stored to vtk file.")

try:
    meshn = pv.read(outdir+ind+'_extracted_borders.vtk')
    p4 = pv.Plotter()
    p4.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
    p4.add_mesh(meshn, point_size=6, color="m")
    p4.camera_position = p4_camera_position
    p4.show()
except:
    print("No Borders identified that can be loaded from vtk file.")

In [ ]:
smooth_mesh_qT1_im.translate((-0.015,0,0))

#plot surface
p3 = pv.Plotter()
p3.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff",show_scalar_bar=False)
p3.add_mesh(mesh_f, opacity='linear', cmap=my_colormap_d1, show_scalar_bar=False)
p3.add_mesh(mesh_h, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)
p3.add_mesh(test_yh_red, point_size=8, color='r')
p3.add_mesh(test_yf_red,point_size=8, color='c')
p3.add_mesh(test_yd5_red,point_size=8, color='y')
#p3.add_mesh(mesh_d5, opacity='linear', cmap=my_colormap_d5, show_scalar_bar=False)

for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p3.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=9, color = crgb[k])
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
p3.camera_position = p4_camera_position
p3.camera.zoom(2.0)
#p4.enable_anti_aliasing('msaa')
#p4.camera.view_frustum(1.0)
#p4.camera.view_angle = 20
p3.show()

#save surface plot
p3 = pv.Plotter(off_screen=True)
p3.set_background('white')
p3.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff", show_scalar_bar=False)
p3.add_mesh(mesh_f, opacity='linear', cmap=my_colormap_d1, show_scalar_bar=False)
p3.add_mesh(mesh_h, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)
#p3.add_mesh(mesh_d5, opacity='linear', cmap=my_colormap_d5, show_scalar_bar=False)
for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p3.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=10, color = crgb[k])
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
p3.camera_position = p4_camera_position
p3.camera.zoom(2.0)
p3.screenshot(outdir+ind+'_qT1_signal_final_borders_smooth_zoomed_pRF.png', window_size=(500,950))

In [ ]:
smooth_mesh_qT1_im.translate((-0.015,0,0))

#plot surface
p2 = pv.Plotter()
p2.set_background('white')
p2.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff",show_scalar_bar=False)
p2.add_mesh(mesh_f, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)
p2.add_mesh(mesh_h, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)

for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p2.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=9, color = "white")
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
p2.camera_position = p4_camera_position
p2.camera.zoom(2.0)
#p4.enable_anti_aliasing('msaa')
#p4.camera.view_frustum(1.0)
#p4.camera.view_angle = 20
p2.show()

#save surface plot
p2 = pv.Plotter(off_screen=True)
p2.set_background('white')
p2.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff", show_scalar_bar=False)
p2.add_mesh(mesh_f, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)
p2.add_mesh(mesh_h, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)

for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p2.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=10, color = "white")
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
p2.camera_position = p4_camera_position
p2.camera.zoom(2.0)
p2.screenshot(outdir+ind+'_qT1_signal_final_borders_smooth_zoomed_pRF_white.png', window_size=(500,950))

In [ ]:
smooth_mesh_qT1_im.translate((-0.015,0,0))

#plot surface
p2 = pv.Plotter()
p2.set_background('white')
p2.add_mesh(mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff",show_scalar_bar=False)
p2.add_mesh(mesh_f, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)
p2.add_mesh(mesh_h, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)

for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p2.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=9, color = "white")
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
p2.camera_position = p4_camera_position
p2.camera.zoom(2.0)
#p4.enable_anti_aliasing('msaa')
#p4.camera.view_frustum(1.0)
#p4.camera.view_angle = 20
p2.show()

#save surface plot
p2 = pv.Plotter(off_screen=True)
p2.set_background('white')
p2.add_mesh(mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff", show_scalar_bar=False)
p2.add_mesh(mesh_f, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)
p2.add_mesh(mesh_h, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)

for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p2.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=10, color = "white")
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
p2.camera_position = p4_camera_position
p2.camera.zoom(2.0)
p2.screenshot(outdir+ind+'_qT1_signal_final_borders_orig_zoomed_pRF_white.png', window_size=(500,950))

In [ ]:
smooth_mesh_qT1_im.translate((-0.015,0,0))

#plot surface
p = pv.Plotter()
p.set_background('white')
p.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff",show_scalar_bar=False)
p.add_mesh(mesh_f, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)
p.add_mesh(mesh_h, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)

p.camera_position = p4_camera_position
p.camera.zoom(2.0)
#p4.enable_anti_aliasing('msaa')
#p4.camera.view_frustum(1.0)
#p4.camera.view_angle = 20
p.show()

#save surface plot
p = pv.Plotter(off_screen=True)
p.set_background('white')
p.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff", show_scalar_bar=False)
p.add_mesh(mesh_f, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)
p.add_mesh(mesh_h, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)

p.camera_position = p4_camera_position
p.camera.zoom(2.0)
p.screenshot(outdir+ind+'_qT1_signal_pRF.png', window_size=(500,950))

In [ ]:
#plot surface
p5 = pv.Plotter()
p5.set_background('white')
p5.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p5.camera_position = p4_camera_position
p5.show()

#plot surface
p5 = pv.Plotter()
p5.set_background('white')
p5.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p5.camera_position = p4_camera_position
p5.screenshot(outdir+ind+'_qT1_signal_smooth.png')

#plot surface
p5 = pv.Plotter()
p5.set_background('white')
p5.add_mesh(smooth_mesh_qT1_im, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff")
p5.camera_position = p4_camera_position
p5.camera.zoom(2.0)
p5.screenshot(outdir+ind+'_qT1_signal_smooth_zoomed.png', window_size=(500,950))

In [ ]:
#plot surface
p3 = pv.Plotter()
p3.set_background('white')
try:
    p3.add_mesh(mesh_h, cmap="RdPu", clim=[0, mesh_h["EmbedVertex"].max()],show_scalar_bar=False)
except:
    print("No motor localizer.")
    
#for k in range(len(border_loc_mesh)):
#    for i in range(0,len(border_loc_mesh[k])):
#        p3.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=9, color = crgb[k])
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
p3.camera_position = p4_camera_position
p3.camera.zoom(2.0)
p3.show()

#save surface plot
p3 = pv.Plotter(off_screen=True)
p3.set_background('white')
try:
    p3.add_mesh(mesh_h, cmap="RdPu", clim=[0, mesh_h["EmbedVertex"].max()],show_scalar_bar=False)
except:
    print("No motor localizer.")
    
for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p3.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=10, color = crgb[k])
        
p3.camera_position = p4_camera_position
p3.camera.zoom(2.0)
p3.screenshot(outdir+ind+'_qT1_signal_final_borders_smooth_zoomed_motorloc_h.png', window_size=(500,950))

In [ ]:
#plot surface
p3 = pv.Plotter()
p3.set_background('white')
try:
    p3.add_mesh(mesh_h, cmap="RdPu", clim=[0, mesh_h["EmbedVertex"].max()],show_scalar_bar=False)
except:
    print("No motor localizer.")
    
for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p3.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=9, color = "black")
        
p3.camera_position = p4_camera_position
p3.camera.zoom(2.0)
p3.show()

#save surface plot
p3 = pv.Plotter(off_screen=True)
p3.set_background('white')
try:
    p3.add_mesh(mesh_h, cmap="RdPu", clim=[0, mesh_h["EmbedVertex"].max()],show_scalar_bar=False)
except:
    print("No motor localizer.")
    
for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p3.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=10, color = "black")
        
p3.camera_position = p4_camera_position
p3.camera.zoom(2.0)
p3.screenshot(outdir+ind+'_qT1_signal_final_borders_smooth_zoomed_motorloc_h_black.png', window_size=(500,950))

In [ ]:
# generate binary BA3b mask #
tmp = mesh_qT1_im
tmp0 = tmp['EmbedVertex']
tmp0[tmp0>0] = 1
tmp['EmbedVertex'] = tmp0

bg=smooth_mesh_qT1_im
bgv=bg['EmbedVertex']
bgv[bgv>0] = 0
bg['EmbedVertex']=bgv

#plot surface
p0 = pv.Plotter()
p0.set_background('white')
p0.add_mesh(bg, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff",show_scalar_bar=False)
p0.add_mesh(mesh_f, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)
p0.add_mesh(mesh_h, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)

for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p0.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=9, color = crgb[k])
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])

p0.camera_position = p4_camera_position
p0.camera.zoom(2.0)
#p4.enable_anti_aliasing('msaa')
#p4.camera.view_frustum(1.0)
#p4.camera.view_angle = 20
p0.show()

#save surface plot
p0 = pv.Plotter(off_screen=True)
p0.set_background('white')
p0.add_mesh(bg, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff", show_scalar_bar=False)
p0.add_mesh(mesh_f, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)
p0.add_mesh(mesh_h, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)

for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p0.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=10, color = crgb[k])
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])

p0.camera_position = p4_camera_position
p0.camera.zoom(2.0)
p0.screenshot(outdir+ind+'_qT1_signal_final_borders_smooth_zoomed_pRF_plain.png', window_size=(500,950))

In [ ]:
#plot surface
p0 = pv.Plotter()
p0.set_background('white')
p0.add_mesh(bg, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff",show_scalar_bar=False)
p0.add_mesh(mesh_d1, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)
p0.add_mesh(mesh_d2, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)

for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p0.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=9, color = "black")
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])

p0.camera_position = p4_camera_position
p0.camera.zoom(2.0)
#p4.enable_anti_aliasing('msaa')
#p4.camera.view_frustum(1.0)
#p4.camera.view_angle = 20
p0.show()

#save surface plot
p0 = pv.Plotter(off_screen=True)
p0.set_background('white')
p0.add_mesh(bg, cmap=qT1_cmap, clim=[mean_im-(4*sd_im), mean_im+(4*sd_im)],below_color="#ffffff", show_scalar_bar=False)
p0.add_mesh(mesh_d1, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)
p0.add_mesh(mesh_d2, opacity='linear', cmap=my_colormap_d2, show_scalar_bar=False)

for k in range(len(border_loc_mesh)):
    for i in range(0,len(border_loc_mesh[k])):
        p0.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=10, color = "black")
        #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])

p0.camera_position = p4_camera_position
p0.camera.zoom(2.0)
p0.screenshot(outdir+ind+'_qT1_signal_final_borders_smooth_zoomed_pRF_plain_black.png', window_size=(500,950))

In [ ]:
# load QSM WB
if subj == 0:
    folder_qsm = 'dummy_data/one_hander/'+ind+'_middle_sQSM_left_orig.vtk'
    folder_pqsm = 'dummy_data/one_hander/'+ind+'_middle_pQSM_left_orig.vtk'
    folder_nqsm = 'dummy_data/one_hander/'+ind+'_middle_nQSM_left_orig.vtk'
else:
    folder_qsm = 'dummy_data/one_hander/'+ind+'_middle_sQSM_right_orig.vtk'
    folder_pqsm = 'dummy_data/one_hander/'+ind+'_middle_pQSM_right_orig.vtk'
    folder_nqsm = 'dummy_data/one_hander/'+ind+'_middle_nQSM_right_orig.vtk'

mesh_qsm = pv.PolyData()
mesh_pqsm = pv.PolyData()
mesh_nqsm = pv.PolyData()

try:
    mesh_qsm = pv.read(folder_qsm)
    mesh_pqsm = pv.read(folder_pqsm)
    mesh_nqsm = pv.read(folder_nqsm)
    
    qsm_bool = True
    
except:
    print("No QSM Data available.")
    
    qsm_bool = False
    
if qsm_bool:
    # plot mesh #
    p = pv.Plotter()
    p.set_background('white')
    p.add_mesh(mesh_qsm, cmap='PuOr', clim=[-0.05, 0.05],below_color="#ffffff",show_scalar_bar=False)
    p.camera_position = p4_camera_position
    p.show()
    
    # apply binary BA3b mask to maps #
    tmp1=mesh_qsm
    qsm=tmp1['EmbedVertex']*tmp['EmbedVertex']
    mesh_qsm['EmbedVertex']=qsm

In [ ]:
mesh_idx_all = []
embed_vert_qsm_all = []
embed_vert_pqsm_all = []
embed_vert_nqsm_all = []

embed_vert_h_all = []
embed_vert_f_all = []
embed_vert_d5_all = []

mesh_points_str = []
mesh_points_str_repl = []

mesh_points_str = list(map(str,mesh_qT1_im.points))
mesh_points_str_repl = [sub.replace(" ", "") for sub in mesh_points_str]

for k in range(len(border_loc_mesh)):
    mesh_idx = []
    emb_vert_qsm = []
    emb_vert_pqsm = []
    emb_vert_nqsm = []
    emb_vert_h = []
    emb_vert_f = []
    emb_vert_d5 = []
    
    for i in range(0,len(border_loc_mesh[k])):
        target_location = str(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])
        target_location1 = target_location.replace(" ", "")
        print(target_location1)

        try:
            print(mesh_points_str_repl.index(target_location1))
            mesh_idx.append(mesh_points_str_repl.index(target_location1))
        except:
            print("Value not in list.")
            
        try:
            emb_vert_qsm.append(mesh_qsm['EmbedVertex'][mesh_points_str_repl.index(target_location1)])
            emb_vert_pqsm.append(mesh_pqsm['EmbedVertex'][mesh_points_str_repl.index(target_location1)])
            emb_vert_nqsm.append(mesh_nqsm['EmbedVertex'][mesh_points_str_repl.index(target_location1)])
        except:
            print("No QSM Data.")
        
        try:
             emb_vert_ecm.append(mesh_ecm['EmbedVertex'][mesh_points_str_repl.index(target_location1)])
        except:
            print("No ECM Data.")
            
        try:
            emb_vert_f.append(mesh_f['EmbedVertex'][mesh_points_str_repl.index(target_location1)])
        except:
            print("No f Map.")
            
        try:
            emb_vert_h.append(mesh_h['EmbedVertex'][mesh_points_str_repl.index(target_location1)])
        except:
            print("No h Map.")
            
    
    mesh_idx_all.append(mesh_idx)
    embed_vert_qsm_all.append(emb_vert_qsm)
    embed_vert_pqsm_all.append(emb_vert_pqsm)
    embed_vert_nqsm_all.append(emb_vert_nqsm)
    embed_vert_f_all.append(emb_vert_f)
    embed_vert_h_all.append(emb_vert_h)
    
print(mesh_idx_all)
print(embed_vert_qsm_all)
print(embed_vert_pqsm_all)
print(embed_vert_nqsm_all)
print(embed_vert_f_all)
print(embed_vert_h_all)

In [ ]:
borderidx = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
borderqsm = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
borderaqsm = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
borderpqsm = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
bordernqsm = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
borderf = np.empty((5,5,)) * np.nan # 5 paths x 5 borders
borderh = np.empty((5,5,)) * np.nan # 5 paths x 5 borders

for j in range(0,len(mesh_idx_all)):
    if len(mesh_idx_all[j]) > 0:
        for k in range(0,len(mesh_idx_all[j])):
            borderidx[j][k] = mesh_idx_all[j][k]      
borderidx = np.array(borderidx).transpose()

for j in range(0,len(embed_vert_qsm_all)):
    if len(embed_vert_qsm_all[j]) > 0:
        for k in range(0,len(embed_vert_qsm_all[j])):
            if embed_vert_qsm_all[j][k] == 0:
                borderqsm[j][k] = np.nan
                borderaqsm[j][k] = np.nan
                borderpqsm[j][k] = np.nan
                bordernqsm[j][k] = np.nan
            else:
                borderqsm[j][k] = embed_vert_qsm_all[j][k]
                borderaqsm[j][k] = abs(embed_vert_qsm_all[j][k])
                borderpqsm[j][k] = embed_vert_pqsm_all[j][k]
                bordernqsm[j][k] = embed_vert_nqsm_all[j][k]

borderqsm = np.array(borderqsm).transpose()
borderaqsm = np.array(borderaqsm).transpose()
borderpqsm = np.array(borderpqsm).transpose()
bordernqsm = np.array(bordernqsm).transpose()

for j in range(0,len(embed_vert_h_all)):
    if len(embed_vert_h_all[j]) > 0:
        #print("test")
        for k in range(0,len(embed_vert_h_all[j])):
            if embed_vert_h_all[j][k] == 0:
                borderh[j][k] = np.nan
            else:
                borderh[j][k] = embed_vert_h_all[j][k]
borderh = np.array(borderh).transpose()

for j in range(0,len(embed_vert_f_all)):
    if len(embed_vert_f_all[j]) > 0:
        for k in range(0,len(embed_vert_f_all[j])):
            if embed_vert_f_all[j][k] == 0:
                borderf[j][k] = np.nan
            else:
                borderf[j][k] = embed_vert_f_all[j][k]
borderf = np.array(borderf).transpose()

print(borderidx)
print(borderqsm)
    
np.savetxt(outdir+ind+"_border_middle_qsm.csv", borderqsm, delimiter=" ")
np.savetxt(outdir+ind+"_border_middle_aqsm.csv", borderaqsm, delimiter=" ")
np.savetxt(outdir+ind+"_border_middle_pqsm.csv", borderpqsm, delimiter=" ")
np.savetxt(outdir+ind+"_border_middle_nqsm.csv", bordernqsm, delimiter=" ")
np.savetxt(outdir+ind+"_border_orig_idx.csv", borderidx, delimiter=" ")

np.savetxt(outdir+ind+"_border_middle_f.csv", borderf, delimiter=" ")
np.savetxt(outdir+ind+"_border_middle_f.csv", borderh, delimiter=" ")

In [ ]:
print(header)
print(border_params)

border_params_allsubj = np.genfromtxt(outdir_all+"allsubj_border_params_middle_multi_y_paths.csv",delimiter=",")
print(border_params_allsubj)
print(border_params_allsubj[ind_i])


mean_sqsm = np.nanmean(borderqsm,axis=0)
print('sqsm')
print(mean_sqsm)

mean_aqsm = np.nanmean(abs(borderqsm),axis=0)
print('aqsm')
print(mean_aqsm)

mean_pqsm = np.nanmean(borderpqsm,axis=0)
print('pqsm')
print(mean_pqsm)

mean_nqsm = np.nanmean(bordernqsm,axis=0)
print('nqsm')
print(mean_nqsm)

mean_f = np.nanmean(borderf,axis=0)
print(mean_f)

mean_h = np.nanmean(borderh,axis=0)
print(mean_h)

for i in range(0,len(mean_sqsm)):
    
    colsqsm = np.where(header == 'borderqsm'+str(i+1))[1]
    print(colsqsm)
    colaqsm = np.where(header == 'borderaqsm'+str(i+1))[1]
    colpqsm = np.where(header == 'borderpqsm'+str(i+1))[1]
    colnqsm = np.where(header == 'bordernqsm'+str(i+1))[1]
    
    border_params_allsubj[ind_i][colsqsm] = mean_sqsm[i]
    border_params_allsubj[ind_i][colaqsm] = mean_aqsm[i]
    border_params_allsubj[ind_i][colpqsm] = mean_pqsm[i]
    border_params_allsubj[ind_i][colnqsm] = mean_nqsm[i]
    
for i in range(0,len(mean_f)):
    
    colf = np.where(header == 'borderspmTf'+str(i+1))[1]
    print(colf)
    
    border_params_allsubj[ind_i][colf] = mean_f[i]
    
for i in range(0,len(mean_h)):
    
    colh = np.where(header == 'borderspmTh'+str(i+1))[1]
    print(colh)
    
    border_params_allsubj[ind_i][colh] = mean_h[i]
    
print(border_params_allsubj[ind_i])

np.savetxt(outdir_all+"allsubj_border_params_middle_multi_y_paths.csv", border_params_allsubj, delimiter=",") 

In [ ]:
#qsm_bool = True

mesh_qsm.translate((-0.015,0,0))
    
if qsm_bool == True:
    
    vtx_qsm = mesh_qsm['EmbedVertex']
    
    # plot ecm together with borders
    p0 = pv.Plotter()
    p0.set_background('white')
    p0.add_mesh(mesh_qsm, cmap='PuOr', clim=[-0.05, 0.05],below_color="#ffffff",show_scalar_bar=True)

    for k in range(len(border_loc_mesh)):
        for i in range(0,len(border_loc_mesh[k])):
            p0.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=9, color = crgb[k])
            #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])

    p0.camera_position = p4_camera_position
    p0.camera.zoom(2.0)
    #p4.enable_anti_aliasing('msaa')
    #p4.camera.view_frustum(1.0)
    #p4.camera.view_angle = 20
    p0.show()

    #save surface plot
    p0 = pv.Plotter(off_screen=True)
    p0.set_background('white')
    p0.add_mesh(mesh_qsm, cmap='PuOr', clim=[-0.05, 0.05],below_color="#ffffff", show_scalar_bar=True)

    for k in range(len(border_loc_mesh)):
        for i in range(0,len(border_loc_mesh[k])):
            p0.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=10, color = crgb[k])
            #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])

    p0.camera_position = p4_camera_position
    p0.camera.zoom(2.0)
    p0.screenshot(outdir+ind+'_qsm_final_borders_smooth_zoomed.png', window_size=(500,950))

    #save surface plot
    p0 = pv.Plotter(off_screen=True)
    p0.set_background('white')
    p0.add_mesh(mesh_qsm, cmap='PuOr', clim=[-0.04, 0.04],below_color="#ffffff", show_scalar_bar=True)

    for k in range(len(border_loc_mesh)):
        for i in range(0,len(border_loc_mesh[k])):
            p0.add_points(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:], point_size=10, color = "white")
            #print(merged_path[border_loc_mesh[k][i][0]].points[border_loc_mesh[k][i][1],:])

    p0.camera_position = p4_camera_position
    p0.camera.zoom(2.0)
    p0.screenshot(outdir+ind+'_qsm_final_borders_smooth_zoomed_white.png', window_size=(500,950))
    
else:
    print("No qsm Data available.")

In [ ]:
print(error_msg)

In [ ]:
print(path_msg)